In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import bisect

import matplotlib.pyplot as plt
%matplotlib inline  

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm

# pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000
# pd.options.display.max_seq_items = 2000

In [43]:
training_df = pd.read_csv('df_log/228/training_self/training_self_0314.csv')
dev_df = pd.read_csv('df_log/228/training_self/dev_self_0314.csv')
test_df = pd.read_csv('df_log/228/training_self/test_self_0314.csv')
training_df = training_df.fillna(0)
dev_df = dev_df.fillna(0)
test_df = test_df.fillna(0)

In [44]:
five_folds = [[[0, 1, 2, 3], [4]], [[0, 1, 2, 4], [3]], [[0, 1, 3, 4], [2]], [[0, 2, 3, 4], [1]], [[1, 2, 3, 4], [0]]]

In [4]:
from functools import reduce

temp_dfs = []

for i in range(0, 5):
    temp_dfs.append(pd.read_csv('from_table/228/training_self/customer_virus_ratio_fold'+str(i)+'.csv').set_index('t1_CustomerID'))

# print(temp_dfs[0].head())
# print(temp_dfs[1].head())
# print(temp_dfs[0].add(temp_dfs[1], fill_value=0).head())

customer_virus_ratio_dfs = []
customer_virus_ratio_df_groups = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    temp_df = reduce(lambda left, right: left.add(right, fill_value=0), \
                     [temp_dfs[merge_folds[0]], temp_dfs[merge_folds[1]], temp_dfs[merge_folds[2]], temp_dfs[merge_folds[3]]])
    temp_df['ratio'] = temp_df['virus_event_count'] / temp_df['total_event_count']
    #print(temp_df[['virus_event_count', 'total_event_count']])
    customer_virus_ratio_dfs.append(temp_df.reset_index())
    customer_virus_ratio_df_groups.append(customer_virus_ratio_dfs[idx].groupby('t1_CustomerID'))
    #print(customer_virus_ratio_dfs[idx])
    print(idx)
    print(fold)
    

temp_dfs = []

for i in range(0, 5):
    temp_dfs.append(pd.read_csv('from_table/228/training_self/product_virus_ratio_training_self_fold'+str(i)+'.csv').set_index('t1_ProductID'))

# print(temp_dfs[0].head())
# print(temp_dfs[1].head())
# print(temp_dfs[0].add(temp_dfs[1], fill_value=0).head())

product_virus_ratio_dfs = []
product_virus_ratio_df_groups = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    temp_df = reduce(lambda left, right: left.add(right, fill_value=0), \
                     [temp_dfs[merge_folds[0]], temp_dfs[merge_folds[1]], temp_dfs[merge_folds[2]], temp_dfs[merge_folds[3]]])
    temp_df['ratio'] = temp_df['virus_event_count'] / temp_df['total_event_count']
    #print(temp_df[['virus_event_count', 'total_event_count']])
    product_virus_ratio_dfs.append(temp_df.reset_index())
    product_virus_ratio_df_groups.append(product_virus_ratio_dfs[idx].groupby('t1_ProductID'))
    #print(product_virus_ratio_dfs[idx])
    print(idx)
    print(fold)
    

temp_dfs = []

for i in range(0, 5):
    customer_virus_ratio_df = pd.read_csv('from_table/228/training_self/customer_virus_ratio_fold'+str(i)+'.csv')
    customer_file_count_df = pd.read_csv('from_table/228/training_self/customer_file_count_training_self_fold'+str(i)+'.csv')
    
    customer_virus_ratio_df['t2_CustomerID'] = customer_virus_ratio_df['t1_CustomerID']
    customer_virus_ratio_df = customer_virus_ratio_df.drop('t1_CustomerID', axis=1)
    
    customer_file_count_df = customer_file_count_df.merge(customer_virus_ratio_df.drop('total_event_count', axis=1), on='t2_CustomerID', how='left')
    customer_file_count_df = customer_file_count_df.fillna(0)
    temp_dfs.append(customer_file_count_df.set_index('t2_CustomerID'))

# print(temp_dfs[0].head())
# print(temp_dfs[1].head())
# print(temp_dfs[0].add(temp_dfs[1], fill_value=0).head())

customer_file_count_dfs = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    temp_df = reduce(lambda left, right: left.add(right, fill_value=0), \
                     [temp_dfs[merge_folds[0]], temp_dfs[merge_folds[1]], temp_dfs[merge_folds[2]], temp_dfs[merge_folds[3]]])
    #print(temp_df[['virus_event_count', 'total_event_count']])
    temp_df['ratio'] = temp_df['virus_event_count'] / (temp_df['total_event_count']+.000000001)
    customer_file_count_dfs.append(temp_df.reset_index())
    #print(product_virus_ratio_dfs[idx])
    print(idx)
    print(fold)

0
[[0, 1, 2, 3], [4]]
1
[[0, 1, 2, 4], [3]]
2
[[0, 1, 3, 4], [2]]
3
[[0, 2, 3, 4], [1]]
4
[[1, 2, 3, 4], [0]]
0
[[0, 1, 2, 3], [4]]
1
[[0, 1, 2, 4], [3]]
2
[[0, 1, 3, 4], [2]]
3
[[0, 2, 3, 4], [1]]
4
[[1, 2, 3, 4], [0]]
0
[[0, 1, 2, 3], [4]]
1
[[0, 1, 2, 4], [3]]
2
[[0, 1, 3, 4], [2]]
3
[[0, 2, 3, 4], [1]]
4
[[1, 2, 3, 4], [0]]


In [4]:
training_five_folds = []
for idx, fold in enumerate(five_folds):
    temp_df = pd.read_csv('df_log/228/training_self/training_self_0314_fold'+str(idx)+'.csv')
    training_five_folds.append(temp_df)

In [5]:
# training_df = pd.concat([training_five_folds[0], training_five_folds[1], training_five_folds[2], training_five_folds[3], training_five_folds[4]])
# training_df.to_csv('df_log/228/training_self/training_self_0314.csv', index=False)

In [8]:
#training_five_folds = []

#for fold_idx, fold in enumerate(five_folds):
for fold_idx, fold in enumerate(training_five_folds):
    customer_virus_ratio_df_group = customer_virus_ratio_df_groups[fold_idx]
    product_virus_ratio_df_group = product_virus_ratio_df_groups[fold_idx]
    customer_file_count_df = customer_file_count_dfs[fold_idx].sort_values('t2_CustomerID')
    customer_file_count_array = customer_file_count_df['t2_CustomerID'].tolist()
    customer_file_count_array.sort()
#     print(customer_file_count_df.head())
#     print(customer_file_count_array[0:10])
    
    #fold_training = training_df[training_df['fold'].isin(fold[1])].copy()
    fold_training = fold.copy()
    
    for data_idx, data in tqdm(fold_training.iterrows(), total=len(fold_training)):
        temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
        unique_customers = temp_df['CustomerID'].unique().tolist()
        unique_products = temp_df['ProductID'].unique().tolist()
        
        temp_ratio = []
        temp_count_index = []
        for customer_id in unique_customers:
            try:
                found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
                if found_ratio['total_event_count'] >= 3:
                    temp_ratio.append(found_ratio['ratio'])
            except:
                temp_ratio.append(0)
                
            try:
                found_count_index = bisect.bisect(customer_file_count_array, customer_id)
                if customer_file_count_array[found_count_index-1] == customer_id:
                    temp_count_index.append(found_count_index-1)
            except:
                pass
                
#         if len(temp_ratio) > 0:        
#             fold_training.loc[data_idx, 'customer_virus_ratio_avg'] = np.mean(temp_ratio)
#             fold_training.loc[data_idx, 'customer_virus_ratio_min'] = np.min(temp_ratio)
#             fold_training.loc[data_idx, 'customer_virus_ratio_max'] = np.max(temp_ratio)
#             fold_training.loc[data_idx, 'customer_virus_ratio_median'] = np.median(temp_ratio)
#             fold_training.loc[data_idx, 'customer_virus_ratio_std'] = np.std(temp_ratio)
#             fold_training.loc[data_idx, 'customer_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#         else:
#             fold_training.loc[data_idx, 'customer_virus_ratio_avg'] = 0
#             fold_training.loc[data_idx, 'customer_virus_ratio_min'] = 0
#             fold_training.loc[data_idx, 'customer_virus_ratio_max'] = 0
#             fold_training.loc[data_idx, 'customer_virus_ratio_median'] = 0
#             fold_training.loc[data_idx, 'customer_virus_ratio_std'] = 0
#             fold_training.loc[data_idx, 'customer_virus_ratio_rstd'] = 0
            
#         temp_ratio = []
#         for product_id in unique_products:
#             try:
#                 found_ratio = product_virus_ratio_df_group.get_group(product_id).iloc[0]
#                 if found_ratio['total_event_count'] >= 3:
#                     temp_ratio.append(found_ratio['ratio'])
#             except:
#                 temp_ratio.append(0)
                
#         if len(temp_ratio) > 0:        
#             fold_training.loc[data_idx, 'product_virus_ratio_avg'] = np.mean(temp_ratio)
#             fold_training.loc[data_idx, 'product_virus_ratio_min'] = np.min(temp_ratio)
#             fold_training.loc[data_idx, 'product_virus_ratio_max'] = np.max(temp_ratio)
#             fold_training.loc[data_idx, 'product_virus_ratio_median'] = np.median(temp_ratio)
#             fold_training.loc[data_idx, 'product_virus_ratio_std'] = np.std(temp_ratio)
#             fold_training.loc[data_idx, 'product_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#         else:
#             fold_training.loc[data_idx, 'product_virus_ratio_avg'] = 0
#             fold_training.loc[data_idx, 'product_virus_ratio_min'] = 0
#             fold_training.loc[data_idx, 'product_virus_ratio_max'] = 0
#             fold_training.loc[data_idx, 'product_virus_ratio_median'] = 0
#             fold_training.loc[data_idx, 'product_virus_ratio_std'] = 0
#             fold_training.loc[data_idx, 'product_virus_ratio_rstd'] = 0
        
        customer_has_static_df = customer_file_count_df.iloc[temp_count_index, :]
        fold_training.loc[data_idx, 'customer_has_static_ratio'] = customer_has_static_df.shape[0] / len(unique_customers)
        customer_has_static_b3_df = customer_has_static_df[customer_has_static_df['total_event_count']>=3]
        fold_training.loc[data_idx, 'customer_has_static_b3_ratio'] = customer_has_static_b3_df.shape[0] / len(unique_customers)

        fold_training.loc[data_idx, 'customer_has_static_count'] = customer_has_static_df.shape[0]
        fold_training.loc[data_idx, 'customer_has_static_b3_count'] = customer_has_static_b3_df.shape[0]

        if customer_has_static_df.shape[0] > 0:
            fold_training.loc[data_idx, 'customer_virus_ratio_wavg'] = np.average(customer_has_static_df['ratio'], weights=customer_has_static_df['total_event_count'])            
            fold_training.loc[data_idx, 'customer_virus_zero_count'] = np.sum(customer_has_static_df['virus_event_count']==0)
            fold_training.loc[data_idx, 'customer_virus_zero_ratio'] = fold_training.loc[data_idx, 'customer_virus_zero_count'] / fold_training.loc[data_idx, 'affect_user_count']
            fold_training.loc[data_idx, 'customer_virus_zero_ratio2'] = fold_training.loc[data_idx, 'customer_virus_zero_count'] / fold_training.loc[data_idx, 'customer_has_static_count']
        
#         print(fold_training.loc[data_idx])
#         break
    
    #training_five_folds.append(fold_training)
    print(fold_training.head())
    fold_training.to_csv('training_self_0314_fold'+str(fold_idx)+'.csv', index=False)
    

100%|██████████| 6742/6742 [03:05<00:00, 36.30it/s]


                             FileID  label  fold  affect_user_count  \
0  cc22fa47ac06f6f0813379b3fe4fa252      1     4                 22   
1  cc2c2dd7bfbfde8e2f0658f27aa5f082      1     4                 15   
2  cc2fbf1cc2671ccb3a6b8443b2d1bf25      1     4                  3   
3  cc3405ec030a2479bc88669333fe8f15      1     4                156   
4  cc39f4ca9e62c6a4edfd79bda5d8da52      1     4                  7   

   total_event_count  each_user_event_avg  each_user_event_std  \
0                 87             3.954545             8.357929   
1                 31             2.066667             2.086236   
2                395           131.666667           114.369285   
3                336             2.153846             5.838862   
4                 31             4.428571             5.191568   

   each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
0              2.113499        1491630510       1492088818         458308   
1              1.00946

100%|██████████| 6694/6694 [03:02<00:00, 36.68it/s]


                             FileID  label  fold  affect_user_count  \
0  98e6408d996cf68b2c18b32e451b6b23      0     3                 23   
1  98e6d86eea7114f4012cc93ac4cfebb5      0     3                 31   
2  98e7d6284ae3903e9ac4f331215121a9      0     3                671   
3  98ea5e3f8cf10f9dea74db353603e6dd      0     3                164   
4  98eb30382fda57947d293ff078cdd440      0     3                642   

   total_event_count  each_user_event_avg  each_user_event_std  \
0                188             8.173913            10.002964   
1                 71             2.290323             1.883145   
2                733             1.092399             0.289805   
3               1889            11.518293            49.377036   
4                642             1.000000             0.000000   

   each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
0              1.223767        1489971944       1490478333         506389   
1              0.82221

100%|██████████| 6738/6738 [03:00<00:00, 37.37it/s]


                             FileID  label  fold  affect_user_count  \
0  65ab4cace270f44f11086cfecefc9ed7      0     2                 62   
1  65adc239f58279f60080bd37a807cd79      0     2                 20   
2  65afdf087f1f54836d7a7bcbb675700a      0     2                  8   
3  65b0615861a338268e67956354e9900d      0     2                 15   
4  65b22629e9c6b7e7ff2f7f131fba07c0      0     2                 20   

   total_event_count  each_user_event_avg  each_user_event_std  \
0                103             1.661290             0.957174   
1                704            35.200000           148.713996   
2                 66             8.250000             7.166390   
3                310            20.666667            29.553019   
4                 83             4.150000             4.233637   

   each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
0              0.576163        1489213092       1489479258         266166   
1              4.22482

100%|██████████| 6684/6684 [03:04<00:00, 36.20it/s]


                             FileID  label  fold  affect_user_count  \
0  32af84f8ff7cd0651650bc74579e409e      0     1                129   
1  32b0b03e84c99675a18e99fe23385b66      0     1                332   
2  32b2ae7cd9903db0a1600d5b041c81e7      0     1                  2   
3  32b4086bbee1f5e725ec512c6f2de0f7      0     1              10092   
4  32b6c20a07a7c83da93c0b4dd17395de      0     1               3868   

   total_event_count  each_user_event_avg  each_user_event_std  \
0                178             1.379845             1.596842   
1                333             1.003012             0.054882   
2                 66            33.000000            32.526912   
3              10092             1.000000             0.000000   
4               3891             1.005946             0.076892   

   each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
0              1.157262        1491610775       1492127688         516913   
1              0.05471

100%|██████████| 6751/6751 [03:02<00:00, 39.45it/s]


                             FileID  label  fold  affect_user_count  \
0  0000e2398b12121a85166fed5fe2a3da      0     0                  3   
1  0001fe8dce14ce099aa6ca8ea5026ea7      0     0                234   
2  00027f50019000accc492e5684efc818      0     0                 22   
3  0003dc8130969abe688cadf5f14ea19f      0     0                130   
4  0006ac9aadbdbf2fed5e62b4e463daa1      0     0                 19   

   total_event_count  each_user_event_avg  each_user_event_std  \
0                 47            15.666667            19.347696   
1                234             1.000000             0.000000   
2                378            17.181818            32.871520   
3                235             1.807692             3.320353   
4                101             5.315789             5.676977   

   each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
0              1.234959        1488608668       1488982004         373336   
1              0.00000

In [8]:
customer_virus_ratio_df = pd.read_csv('from_table/228/training_self/customer_virus_ratio.csv')
customer_virus_ratio_df_group = customer_virus_ratio_df.groupby('t1_CustomerID')
product_virus_ratio_df_group = pd.read_csv('from_table/228/training_self/product_virus_ratio_training_self.csv').groupby('t1_ProductID')

customer_file_count_df = pd.read_csv('from_table/228/training_self/customer_file_count_training_self.csv')

customer_virus_ratio_df_temp = customer_virus_ratio_df.copy()
customer_virus_ratio_df_temp['t2_CustomerID'] = customer_virus_ratio_df_temp['t1_CustomerID']
customer_virus_ratio_df_temp = customer_virus_ratio_df_temp.drop('t1_CustomerID', axis=1)
customer_file_count_df = customer_file_count_df.merge(customer_virus_ratio_df_temp.drop('total_event_count', axis=1), on='t2_CustomerID', how='left')
customer_file_count_df = customer_file_count_df.fillna(0)
customer_file_count_df['ratio'] = customer_file_count_df['virus_event_count'] / (customer_file_count_df['total_event_count']+.000000001)
customer_file_count_df = customer_file_count_df.sort_values('t2_CustomerID')
customer_file_count_df = customer_file_count_df.reset_index()
del customer_virus_ratio_df_temp

print(customer_file_count_df.head())
customer_file_count_array = customer_file_count_df['t2_CustomerID'].tolist()
customer_file_count_array.sort()
print(customer_file_count_array[0:10])

     index                     t2_CustomerID  total_event_count  ratio  \
0   434899  00000145d9062eada528bace5fb4864e                  1    0.0   
1   113980  000003d77bee692d83b7b83fdba60d3d                 24    0.0   
2  1430540  00000aea7887eaf96dbd022025e426fd                  1    0.0   
3  2567972  0000119795c2f643a5111de4a0f135c6                  5    0.0   
4   577427  0000136b39e3e7892157113d03e1cac8                  1    0.0   

   virus_event_count  
0                0.0  
1                0.0  
2                0.0  
3                0.0  
4                0.0  
['00000145d9062eada528bace5fb4864e', '000003d77bee692d83b7b83fdba60d3d', '00000aea7887eaf96dbd022025e426fd', '0000119795c2f643a5111de4a0f135c6', '0000136b39e3e7892157113d03e1cac8', '00001d108011435a6bc742400a8e6abd', '00001e2c535a8954e54f683f13b80d13', '000022386433b9eff82501bd0abd1947', '0000227f85392306dd340588be1dcb21', '000028d95113207e2b0f0c8655f583fa']


In [35]:
import timeit

#customer_virus_ratio_df_big_3 = customer_virus_ratio_df[customer_virus_ratio_df['total_event_count']>=3]
temp_df = pd.read_csv('file_log/'+training_df.iloc[1]['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
unique_customers = temp_df['CustomerID'].unique().tolist()

start = timeit.default_timer()

temp_ratio = []
for customer_id in unique_customers:
        try:
            found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
            if found_ratio['total_event_count'] >= 3:
                temp_ratio.append(found_ratio['ratio'])
        except:
            temp_ratio.append(0)
            
np.mean(temp_ratio)
np.min(temp_ratio)
np.max(temp_ratio)
np.median(temp_ratio)
np.std(temp_ratio)
np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)

stop = timeit.default_timer()
print(stop - start)


start = timeit.default_timer()

customer_virus_ratio_df_group.take(unique_customers)

stop = timeit.default_timer()
print(stop - start)

0.0031375359999401553
0.06211077700004353


In [10]:
training_df.head()

,FileID,label,fold,affect_user_count,total_event_count,each_user_event_avg,each_user_event_std,each_user_event_rstd,first_time_occur,last_time_occur,time_duration,first_24h_count,first_24h_ratio,first_24h_affect_user_count,first_24h_affect_user_ratio,first_24h_each_user_event_avg,bin_0_count,bin_0_ratio,bin_0_affect_user_count,bin_0_affect_user_ratio,bin_0_each_user_event_avg,bin_1_count,bin_1_ratio,bin_1_affect_user_count,bin_1_affect_user_ratio,bin_1_each_user_event_avg,bin_2_count,bin_2_ratio,bin_2_affect_user_count,bin_2_affect_user_ratio,bin_2_each_user_event_avg,bin_3_count,bin_3_ratio,bin_3_affect_user_count,bin_3_affect_user_ratio,bin_3_each_user_event_avg,bin_4_count,bin_4_ratio,bin_4_affect_user_count,bin_4_affect_user_ratio,bin_4_each_user_event_avg,bin_5_count,bin_5_ratio,bin_5_affect_user_count,bin_5_affect_user_ratio,bin_5_each_user_event_avg,bin_6_count,bin_6_ratio,bin_6_affect_user_count,bin_6_affect_user_ratio,bin_6_each_user_event_avg,bin_7_count,bin_7_ratio,bin_7_affect_user_count,bin_7_affect_user_ratio,bin_7_each_user_event_avg,bin_8_count,bin_8_ratio,bin_8_affect_user_count,bin_8_affect_user_ratio,bin_8_each_user_event_avg,bin_9_count,bin_9_ratio,bin_9_affect_user_count,bin_9_affect_user_ratio,bin_9_each_user_event_avg,customer_spread_time_mean,customer_spread_time_std,customer_spread_time_rstd,event_diff_time_mean,event_diff_time_median,event_diff_time_std,event_diff_time_rstd,customer_virus_ratio_avg,customer_virus_ratio_min,customer_virus_ratio_max,customer_virus_ratio_median,customer_virus_ratio_std,customer_virus_ratio_rstd,product_virus_ratio_avg,product_virus_ratio_min,product_virus_ratio_max,product_virus_ratio_median,product_virus_ratio_std,product_virus_ratio_rstd,customer_has_static_ratio,customer_has_static_b3_ratio,customer_has_static_count,customer_has_static_b3_count,customer_virus_ratio_wavg,customer_virus_zero_count,customer_virus_zero_ratio,customer_virus_zero_ratio2,hour_ratio_mean,hour_ratio_std,hour_ratio_max,hour_ratio_min,hour_occupy_ratio,hour_mean
0,cc22fa47ac06f6f0813379b3fe4fa252,1,4,22,87,3.954545,8.357929,2.113499,1491630510,1492088818,458308,11,0.126437,4,0.181818,2.750000,10,0.114943,3,0.136364,3.333333,1,0.011494,1,0.045455,1.00,6,0.068966,4,0.181818,1.50,8,0.091954,1,0.045455,8.000000,6,0.068966,5,0.227273,1.200000,51,0.586207,6,0.272727,8.500000,2,0.022989,2,0.090909,1.000000,1,0.011494,1,0.045455,1.0,0,0.000000,0,0.000000,0.0,2,0.022989,1,0.045455,2.0,15198.000000,15170.078723,0.998163,4993.218391,2.0,37880.054476,7.586300,0.139533,0.000000,1.000000,0.00,0.294399,2.109894,0.117117,0.000000,0.163000,0.135924,0.050167,0.428351,0.545455,0.227273,12.0,5.0,0.156113,0.0,0.000000,0.000000,None,None,None,None,None,None
1,cc2c2dd7bfbfde8e2f0658f27aa5f082,1,4,15,31,2.066667,2.086236,1.009469,1489577110,1489810151,233041,30,0.967742,15,1.000000,2.000000,27,0.870968,14,0.933333,1.928571,2,0.064516,2,0.133333,1.00,0,0.000000,0,0.000000,0.00,1,0.032258,1,0.066667,1.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.0,0,0.000000,0,0.000000,0.0,1,0.032258,1,0.066667,1.0,2273.533333,8554.817981,3.762785,11706.580645,1.0,43469.005530,3.713211,0.930479,0.157413,1.000000,1.00,0.223347,0.240035,0.115556,0.000000,0.163177,0.148015,0.060774,0.525923,1.000000,0.866667,15.0,13.0,0.183113,0.0,0.000000,0.000000,None,None,None,None,None,None
2,cc2fbf1cc2671ccb3a6b8443b2d1bf25,1,4,3,395,131.666667,114.369285,0.868627,1488367460,1488466168,98708,389,0.984810,3,1.000000,129.666667,78,0.197468,1,0.333333,78.000000,251,0.635443,2,0.666667,125.50,60,0.151899,1,0.333333,60.00,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.0,0,0.000000,0,0.000000,0.0,6,0.015190,1,0.333333,6.0,4675.666667,5519.297449,1.180430,246.015190,31.0,4139.997071,16.828217,0.250000,0.000000,0.500000,0.25,0.250000,1.000000,0.065555,0.000000,0.109176,0.087488,0.047

In [5]:
customer_virus_ratio_df.head()

,t1_CustomerID,ratio,virus_event_count,total_event_count
0,715cb911f7a41698d451b350a785a5b2,0.010309,1,97
1,fa4bb2a462899692f85db0833f9fc3e5,1.000000,1,1
2,3f123426716153ddcbba012228868a9a,1.000000,1,1
3,5518cbcdb4c603275d9caf0b6d23974a,0.050000,1,20
4,c6c176669e218000f955932e3c686dc4,0.885246,54,61


In [45]:
# training_df['affect_user_count'] = None
# training_df['total_event_count'] = None
# training_df['each_user_event_avg'] = None
# training_df['each_user_event_std'] = None
# training_df['each_user_event_rstd'] = None

# training_df['first_time_occur'] = None
# training_df['last_time_occur'] = None
# training_df['time_duration'] = None

# training_df['first_24h_count'] = None
# training_df['first_24h_ratio'] = None
# training_df['first_24h_affect_user_count'] = None
# training_df['first_24h_affect_user_ratio'] = None
# training_df['first_24h_each_user_event_avg'] = None

# for i in range(0, 10):
#     training_df['bin_'+str(i)+'_count'] = None
#     training_df['bin_'+str(i)+'_ratio'] = None
#     training_df['bin_'+str(i)+'_affect_user_count'] = None
#     training_df['bin_'+str(i)+'_affect_user_ratio'] = None
#     training_df['bin_'+str(i)+'_each_user_event_avg'] = None

# training_df['customer_spread_time_mean'] = None
# training_df['customer_spread_time_std'] = None
# training_df['customer_spread_time_rstd'] = None

# training_df['event_diff_time_mean'] = None
# training_df['event_diff_time_median'] = None
# training_df['event_diff_time_std'] = None
# training_df['event_diff_time_rstd'] = None

# training_df['customer_virus_ratio_avg'] = None
# training_df['customer_virus_ratio_min'] = None
# training_df['customer_virus_ratio_max'] = None
# training_df['customer_virus_ratio_median'] = None
# training_df['customer_virus_ratio_std'] = None
# training_df['customer_virus_ratio_rstd'] = None    

# training_df['customer_virus_ratio_wavg'] = None
# training_df['customer_virus_zero_count'] = None
# training_df['customer_virus_zero_ratio'] = None
# training_df['customer_virus_zero_ratio2'] = None

training_df['hour_ratio_mean'] = None
training_df['hour_ratio_median'] = None
training_df['hour_ratio_std'] = None
training_df['hour_ratio_rstd'] = None
training_df['hour_ratio_max'] = None
training_df['hour_ratio_min'] = None
training_df['hour_occupy_ratio'] = None
training_df['hour_mean'] = None

training_df['dayofweek_ratio_mean'] = None
training_df['dayofweek_ratio_median'] = None
training_df['dayofweek_ratio_std'] = None
training_df['dayofweek_ratio_rstd'] = None
training_df['dayofweek_ratio_max'] = None
training_df['dayofweek_ratio_min'] = None
training_df['dayofweek_occupy_ratio'] = None
training_df['dayofweek_mean'] = None

training_df['bin_ratio_mean'] = None
training_df['bin_ratio_median'] = None
training_df['bin_ratio_std'] = None
training_df['bin_ratio_rstd'] = None
training_df['bin_ratio_max'] = None
training_df['bin_ratio_min'] = None
training_df['bin_occupy_ratio'] = None

training_df['event_diff_time_mean2'] = None
training_df['event_diff_time_median2'] = None
training_df['event_diff_time_std2'] = None
training_df['event_diff_time_rstd2'] = None
training_df['event_diff_time_max2'] = None
training_df['event_diff_time_min2'] = None
training_df['event_diff_time_lessone_ratio2'] = None

training_df['all_event_diff_time_mean'] = None
training_df['all_event_diff_time_median'] = None
training_df['all_event_diff_time_std'] = None
training_df['all_event_diff_time_rstd'] = None
training_df['all_event_diff_time_max'] = None
training_df['all_event_diff_time_min'] = None
training_df['all_event_diff_time_lessone_ratio'] = None

for idx, data in tqdm(training_df.iterrows(), total=len(training_df)):
    temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
#     total_event_count = len(temp_df)
#     training_df.loc[idx, 'affect_user_count'] = len(temp_df['CustomerID'].unique())
#     training_df.loc[idx, 'total_event_count'] = total_event_count
#     training_df.loc[idx, 'each_user_event_avg'] = training_df.loc[idx, 'total_event_count'] / training_df.loc[idx, 'affect_user_count']
#     training_df.loc[idx, 'each_user_event_std'] = temp_df.groupby('CustomerID').count()['QueryTS'].std()
#     training_df.loc[idx, 'each_user_event_rstd'] = training_df.loc[idx, 'each_user_event_std'] / training_df.loc[idx, 'each_user_event_avg']
    
#     time_start = temp_df['QueryTS'].min()
#     time_end = temp_df['QueryTS'].max()
#     time_range = time_end - time_start
    
#     training_df.loc[idx, 'first_time_occur'] = time_start
#     training_df.loc[idx, 'last_time_occur'] = time_end
#     training_df.loc[idx, 'time_duration'] = time_range
    
#     the_flag = ( (temp_df['QueryTS']>=time_start)&(temp_df['QueryTS']<=time_start+86400) )
#     the_24h_count = np.sum(the_flag)
#     training_df.loc[idx, 'first_24h_count'] = the_24h_count
#     training_df.loc[idx, 'first_24h_ratio'] = the_24h_count / total_event_count
#     training_df.loc[idx, 'first_24h_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#     training_df.loc[idx, 'first_24h_affect_user_ratio'] = training_df.loc[idx, 'first_24h_affect_user_count'] / training_df.loc[idx, 'affect_user_count']
#     training_df.loc[idx, 'first_24h_each_user_event_avg'] = the_24h_count / (training_df.loc[idx, 'first_24h_affect_user_count']+.000000001)
    
#     time_step = time_range/10.0
#     for i in range(0, 10):
#         the_flag = ( (temp_df['QueryTS']>=time_start+time_step*i)&(temp_df['QueryTS']<time_start+time_step*(i+1)+0.5) )
#         the_bin_count = np.sum(the_flag)
#         training_df.loc[idx, 'bin_'+str(i)+'_count'] = the_bin_count
#         training_df.loc[idx, 'bin_'+str(i)+'_ratio'] = the_bin_count / total_event_count
#         training_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#         training_df.loc[idx, 'bin_'+str(i)+'_affect_user_ratio'] = training_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] / training_df.loc[idx, 'affect_user_count']
#         training_df.loc[idx, 'bin_'+str(i)+'_each_user_event_avg'] = the_bin_count / (training_df.loc[idx, 'bin_'+str(i)+'_affect_user_count']+.000000001)
    
#     the_static_df = temp_df.groupby('CustomerID').min().sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().fillna(0)
#     training_df.loc[idx, 'customer_spread_time_mean'] = the_static_df.mean()
#     training_df.loc[idx, 'customer_spread_time_std'] = the_static_df.std()
#     training_df.loc[idx, 'customer_spread_time_rstd'] = training_df.loc[idx, 'customer_spread_time_std'] / (training_df.loc[idx, 'customer_spread_time_mean']+.000000001)
    
#     the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().fillna(0)
#     training_df.loc[idx, 'event_diff_time_mean'] = the_static_df.mean()
#     training_df.loc[idx, 'event_diff_time_median'] = the_static_df.median()
#     training_df.loc[idx, 'event_diff_time_std'] = the_static_df.std()
#     training_df.loc[idx, 'event_diff_time_rstd'] = training_df.loc[idx, 'event_diff_time_std'] / (training_df.loc[idx, 'event_diff_time_mean']+.000000001)
    
#     unique_customers = temp_df['CustomerID'].unique().tolist()
#     unique_products = temp_df['ProductID'].unique().tolist()
    
#     temp_ratio = []
#     temp_count_index = []
#     for customer_id in unique_customers:
#         try:
#             found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)
            
#         try:
#             found_count_index = bisect.bisect(customer_file_count_array, customer_id)
#             if customer_file_count_array[found_count_index-1] == customer_id:
#                 temp_count_index.append(found_count_index-1)
#         except:
#             pass
                   
#     if len(temp_ratio) > 0:        
#         training_df.loc[idx, 'customer_virus_ratio_avg'] = np.mean(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_min'] = np.min(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_max'] = np.max(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_median'] = np.median(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_std'] = np.std(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         training_df.loc[idx, 'customer_virus_ratio_avg'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_min'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_max'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_median'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_std'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_rstd'] = 0

#     temp_ratio = []
#     for product_id in unique_products:
#         try:
#             found_ratio = product_virus_ratio_df_group.get_group(product_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)

#     if len(temp_ratio) > 0:
#         training_df.loc[idx, 'product_virus_ratio_avg'] = np.mean(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_min'] = np.min(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_max'] = np.max(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_median'] = np.median(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_std'] = np.std(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         training_df.loc[idx, 'product_virus_ratio_avg'] = 0
#         training_df.loc[idx, 'product_virus_ratio_min'] = 0
#         training_df.loc[idx, 'product_virus_ratio_max'] = 0
#         training_df.loc[idx, 'product_virus_ratio_median'] = 0
#         training_df.loc[idx, 'product_virus_ratio_std'] = 0
#         training_df.loc[idx, 'product_virus_ratio_rstd'] = 0

#     customer_has_static_df = customer_file_count_df.iloc[temp_count_index, :]
#     training_df.loc[idx, 'customer_has_static_ratio'] = customer_has_static_df.shape[0] / len(unique_customers)
#     customer_has_static_b3_df = customer_has_static_df[customer_has_static_df['total_event_count']>=3]
#     training_df.loc[idx, 'customer_has_static_b3_ratio'] = customer_has_static_b3_df.shape[0] / len(unique_customers)

#     training_df.loc[idx, 'customer_has_static_count'] = customer_has_static_df.shape[0]
#     training_df.loc[idx, 'customer_has_static_b3_count'] = customer_has_static_b3_df.shape[0]

#     if customer_has_static_df.shape[0] > 0:
#         training_df.loc[idx, 'customer_virus_ratio_wavg'] = np.average(customer_has_static_df['ratio'], weights=customer_has_static_df['total_event_count'])            
#         training_df.loc[idx, 'customer_virus_zero_count'] = np.sum(customer_has_static_df['virus_event_count']==0)
#         training_df.loc[idx, 'customer_virus_zero_ratio'] = training_df.loc[idx, 'customer_virus_zero_count'] / training_df.loc[idx, 'affect_user_count']
#         training_df.loc[idx, 'customer_virus_zero_ratio2'] = training_df.loc[idx, 'customer_virus_zero_count'] / training_df.loc[idx, 'customer_has_static_count']

    temp_df['datetime'] = pd.to_datetime(temp_df['QueryTS'], unit='s')
    temp_df['hour'] = temp_df['datetime'].dt.hour
    temp_df['dayofweek'] = temp_df['datetime'].dt.dayofweek
    
    hour_ratio_df = temp_df[['FileID', 'hour']].groupby('hour').count()/temp_df.shape[0]
    training_df.loc[idx, 'hour_ratio_mean'] = hour_ratio_df['FileID'].mean()
    training_df.loc[idx, 'hour_ratio_median'] = hour_ratio_df['FileID'].median()
    training_df.loc[idx, 'hour_ratio_std'] = hour_ratio_df['FileID'].std()
    training_df.loc[idx, 'hour_ratio_rstd'] = hour_ratio_df['FileID'].std() / (hour_ratio_df['FileID'].mean()+.000000001)
    training_df.loc[idx, 'hour_ratio_max'] = hour_ratio_df['FileID'].max()
    training_df.loc[idx, 'hour_ratio_min'] = hour_ratio_df['FileID'].min()
    training_df.loc[idx, 'hour_occupy_ratio'] = temp_df[['FileID', 'hour']].groupby('hour').count().shape[0]/24.0
    training_df.loc[idx, 'hour_mean'] = temp_df['hour'].mean()
    
    dayofweek_ratio_df = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count()/temp_df.shape[0]
    training_df.loc[idx, 'dayofweek_ratio_mean'] = dayofweek_ratio_df['FileID'].mean()
    training_df.loc[idx, 'dayofweek_ratio_median'] = dayofweek_ratio_df['FileID'].median()
    training_df.loc[idx, 'dayofweek_ratio_std'] = dayofweek_ratio_df['FileID'].std()
    training_df.loc[idx, 'dayofweek_ratio_rstd'] = dayofweek_ratio_df['FileID'].std() / (dayofweek_ratio_df['FileID'].mean()+.000000001)
    training_df.loc[idx, 'dayofweek_ratio_max'] = dayofweek_ratio_df['FileID'].max()
    training_df.loc[idx, 'dayofweek_ratio_min'] = dayofweek_ratio_df['FileID'].min()
    training_df.loc[idx, 'dayofweek_occupy_ratio'] = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count().shape[0]/7.0
    training_df.loc[idx, 'dayofweek_mean'] = temp_df['dayofweek'].mean()
    
    bin_ratio_array = data[['bin_0_ratio', 'bin_1_ratio', 'bin_2_ratio', 'bin_3_ratio', 'bin_4_ratio', \
                            'bin_5_ratio', 'bin_6_ratio', 'bin_7_ratio', 'bin_8_ratio', 'bin_9_ratio']].values
    training_df.loc[idx, 'bin_ratio_mean'] = np.mean(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_median'] = np.median(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_std'] = np.std(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_rstd'] = np.std(bin_ratio_array) / (np.mean(bin_ratio_array)+.000000001)
    training_df.loc[idx, 'bin_ratio_max'] = np.max(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_min'] = np.min(bin_ratio_array)
    training_df.loc[idx, 'bin_occupy_ratio'] = np.sum(bin_ratio_array>0)/10.0

    the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        training_df.loc[idx, 'event_diff_time_mean2'] = the_static_df.mean()
        training_df.loc[idx, 'event_diff_time_median2'] = the_static_df.median()
        training_df.loc[idx, 'event_diff_time_std2'] = the_static_df.std()
        training_df.loc[idx, 'event_diff_time_rstd2'] = training_df.loc[idx, 'event_diff_time_std2'] / (training_df.loc[idx, 'event_diff_time_mean2']+.000000001)
        training_df.loc[idx, 'event_diff_time_max2'] = the_static_df.max()
        training_df.loc[idx, 'event_diff_time_min2'] = the_static_df.min()
        training_df.loc[idx, 'event_diff_time_lessone_ratio2'] = np.sum(the_static_df<=1) / len(the_static_df)
    
    the_static_df = temp_df.sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        training_df.loc[idx, 'all_event_diff_time_mean'] = the_static_df.mean()
        training_df.loc[idx, 'all_event_diff_time_median'] = the_static_df.median()
        training_df.loc[idx, 'all_event_diff_time_std'] = the_static_df.std()
        training_df.loc[idx, 'all_event_diff_time_rstd'] = training_df.loc[idx, 'all_event_diff_time_std'] / (training_df.loc[idx, 'all_event_diff_time_mean']+.000000001)
        training_df.loc[idx, 'all_event_diff_time_max'] = the_static_df.max()
        training_df.loc[idx, 'all_event_diff_time_min'] = the_static_df.min()
        training_df.loc[idx, 'all_event_diff_time_lessone_ratio'] = np.sum(the_static_df<=1) / len(the_static_df)
    
#     print(training_df.loc[idx])
# #     print(temp_df)
# #     print(len(temp_df['CustomerID'].unique()))
# #     print(data)
#     if idx==1:
#         break


100%|██████████| 33609/33609 [1:34:51<00:00,  6.78it/s]  


In [46]:
training_df.to_csv('training_self_0323.csv', index=False)

In [47]:
# dev_df['affect_user_count'] = None
# dev_df['total_event_count'] = None
# dev_df['each_user_event_avg'] = None
# dev_df['each_user_event_std'] = None
# dev_df['each_user_event_rstd'] = None

# dev_df['first_time_occur'] = None
# dev_df['last_time_occur'] = None
# dev_df['time_duration'] = None

# dev_df['first_24h_count'] = None
# dev_df['first_24h_ratio'] = None
# dev_df['first_24h_affect_user_count'] = None
# dev_df['first_24h_affect_user_ratio'] = None
# dev_df['first_24h_each_user_event_avg'] = None

# for i in range(0, 10):
#     dev_df['bin_'+str(i)+'_count'] = None
#     dev_df['bin_'+str(i)+'_ratio'] = None
#     dev_df['bin_'+str(i)+'_affect_user_count'] = None
#     dev_df['bin_'+str(i)+'_affect_user_ratio'] = None
#     dev_df['bin_'+str(i)+'_each_user_event_avg'] = None

# dev_df['customer_spread_time_mean'] = None
# dev_df['customer_spread_time_std'] = None
# dev_df['customer_spread_time_rstd'] = None

# dev_df['event_diff_time_mean'] = None
# dev_df['event_diff_time_median'] = None
# dev_df['event_diff_time_std'] = None
# dev_df['event_diff_time_rstd'] = None

# dev_df['customer_virus_ratio_avg'] = None
# dev_df['customer_virus_ratio_min'] = None
# dev_df['customer_virus_ratio_max'] = None
# dev_df['customer_virus_ratio_median'] = None
# dev_df['customer_virus_ratio_std'] = None
# dev_df['customer_virus_ratio_rstd'] = None    

# dev_df['customer_virus_ratio_wavg'] = None
# dev_df['customer_virus_zero_count'] = None
# dev_df['customer_virus_zero_ratio'] = None
# dev_df['customer_virus_zero_ratio2'] = None

dev_df['hour_ratio_mean'] = None
dev_df['hour_ratio_median'] = None
dev_df['hour_ratio_std'] = None
dev_df['hour_ratio_rstd'] = None
dev_df['hour_ratio_max'] = None
dev_df['hour_ratio_min'] = None
dev_df['hour_occupy_ratio'] = None
dev_df['hour_mean'] = None

dev_df['dayofweek_ratio_mean'] = None
dev_df['dayofweek_ratio_median'] = None
dev_df['dayofweek_ratio_std'] = None
dev_df['dayofweek_ratio_rstd'] = None
dev_df['dayofweek_ratio_max'] = None
dev_df['dayofweek_ratio_min'] = None
dev_df['dayofweek_occupy_ratio'] = None
dev_df['dayofweek_mean'] = None

dev_df['bin_ratio_mean'] = None
dev_df['bin_ratio_median'] = None
dev_df['bin_ratio_std'] = None
dev_df['bin_ratio_rstd'] = None
dev_df['bin_ratio_max'] = None
dev_df['bin_ratio_min'] = None
dev_df['bin_occupy_ratio'] = None

dev_df['event_diff_time_mean2'] = None
dev_df['event_diff_time_median2'] = None
dev_df['event_diff_time_std2'] = None
dev_df['event_diff_time_rstd2'] = None
dev_df['event_diff_time_max2'] = None
dev_df['event_diff_time_min2'] = None
dev_df['event_diff_time_lessone_ratio2'] = None

dev_df['all_event_diff_time_mean'] = None
dev_df['all_event_diff_time_median'] = None
dev_df['all_event_diff_time_std'] = None
dev_df['all_event_diff_time_rstd'] = None
dev_df['all_event_diff_time_max'] = None
dev_df['all_event_diff_time_min'] = None
dev_df['all_event_diff_time_lessone_ratio'] = None

for idx, data in tqdm(dev_df.iterrows(), total=len(dev_df)):
    temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
#     total_event_count = len(temp_df)
#     dev_df.loc[idx, 'affect_user_count'] = len(temp_df['CustomerID'].unique())
#     dev_df.loc[idx, 'total_event_count'] = total_event_count
#     dev_df.loc[idx, 'each_user_event_avg'] = dev_df.loc[idx, 'total_event_count'] / dev_df.loc[idx, 'affect_user_count']
#     dev_df.loc[idx, 'each_user_event_std'] = temp_df.groupby('CustomerID').count()['QueryTS'].std()
#     dev_df.loc[idx, 'each_user_event_rstd'] = dev_df.loc[idx, 'each_user_event_std'] / dev_df.loc[idx, 'each_user_event_avg']
    
#     time_start = temp_df['QueryTS'].min()
#     time_end = temp_df['QueryTS'].max()
#     time_range = time_end - time_start
    
#     dev_df.loc[idx, 'first_time_occur'] = time_start
#     dev_df.loc[idx, 'last_time_occur'] = time_end
#     dev_df.loc[idx, 'time_duration'] = time_range
    
#     the_flag = ( (temp_df['QueryTS']>=time_start)&(temp_df['QueryTS']<=time_start+86400) )
#     the_24h_count = np.sum(the_flag)
#     dev_df.loc[idx, 'first_24h_count'] = the_24h_count
#     dev_df.loc[idx, 'first_24h_ratio'] = the_24h_count / total_event_count
#     dev_df.loc[idx, 'first_24h_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#     dev_df.loc[idx, 'first_24h_affect_user_ratio'] = dev_df.loc[idx, 'first_24h_affect_user_count'] / dev_df.loc[idx, 'affect_user_count']
#     dev_df.loc[idx, 'first_24h_each_user_event_avg'] = the_24h_count / (dev_df.loc[idx, 'first_24h_affect_user_count']+.000000001)
    
#     time_step = time_range/10.0
#     for i in range(0, 10):
#         the_flag = ( (temp_df['QueryTS']>=time_start+time_step*i)&(temp_df['QueryTS']<time_start+time_step*(i+1)+0.5) )
#         the_bin_count = np.sum(the_flag)
#         dev_df.loc[idx, 'bin_'+str(i)+'_count'] = the_bin_count
#         dev_df.loc[idx, 'bin_'+str(i)+'_ratio'] = the_bin_count / total_event_count
#         dev_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#         dev_df.loc[idx, 'bin_'+str(i)+'_affect_user_ratio'] = dev_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] / dev_df.loc[idx, 'affect_user_count']
#         dev_df.loc[idx, 'bin_'+str(i)+'_each_user_event_avg'] = the_bin_count / (dev_df.loc[idx, 'bin_'+str(i)+'_affect_user_count']+.000000001)
    
#     the_static_df = temp_df.groupby('CustomerID').min().sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().fillna(0)
#     dev_df.loc[idx, 'customer_spread_time_mean'] = the_static_df.mean()
#     dev_df.loc[idx, 'customer_spread_time_std'] = the_static_df.std()
#     dev_df.loc[idx, 'customer_spread_time_rstd'] = dev_df.loc[idx, 'customer_spread_time_std'] / (dev_df.loc[idx, 'customer_spread_time_mean']+.000000001)
    
#     the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().fillna(0)
#     dev_df.loc[idx, 'event_diff_time_mean'] = the_static_df.mean()
#     dev_df.loc[idx, 'event_diff_time_median'] = the_static_df.median()
#     dev_df.loc[idx, 'event_diff_time_std'] = the_static_df.std()
#     dev_df.loc[idx, 'event_diff_time_rstd'] = dev_df.loc[idx, 'event_diff_time_std'] / (dev_df.loc[idx, 'event_diff_time_mean']+.000000001)
    
#     unique_customers = temp_df['CustomerID'].unique().tolist()
#     unique_products = temp_df['ProductID'].unique().tolist()
    
#     temp_ratio = []
#     temp_count_index = []
#     for customer_id in unique_customers:
#         try:
#             found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)
            
#         try:
#             found_count_index = bisect.bisect(customer_file_count_array, customer_id)
#             if customer_file_count_array[found_count_index-1] == customer_id:
#                 temp_count_index.append(found_count_index-1)
#         except:
#             pass
                   
#     if len(temp_ratio) > 0:        
#         dev_df.loc[idx, 'customer_virus_ratio_avg'] = np.mean(temp_ratio)
#         dev_df.loc[idx, 'customer_virus_ratio_min'] = np.min(temp_ratio)
#         dev_df.loc[idx, 'customer_virus_ratio_max'] = np.max(temp_ratio)
#         dev_df.loc[idx, 'customer_virus_ratio_median'] = np.median(temp_ratio)
#         dev_df.loc[idx, 'customer_virus_ratio_std'] = np.std(temp_ratio)
#         dev_df.loc[idx, 'customer_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         dev_df.loc[idx, 'customer_virus_ratio_avg'] = 0
#         dev_df.loc[idx, 'customer_virus_ratio_min'] = 0
#         dev_df.loc[idx, 'customer_virus_ratio_max'] = 0
#         dev_df.loc[idx, 'customer_virus_ratio_median'] = 0
#         dev_df.loc[idx, 'customer_virus_ratio_std'] = 0
#         dev_df.loc[idx, 'customer_virus_ratio_rstd'] = 0

#     temp_ratio = []
#     for product_id in unique_products:
#         try:
#             found_ratio = product_virus_ratio_df_group.get_group(product_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)

#     if len(temp_ratio) > 0:
#         dev_df.loc[idx, 'product_virus_ratio_avg'] = np.mean(temp_ratio)
#         dev_df.loc[idx, 'product_virus_ratio_min'] = np.min(temp_ratio)
#         dev_df.loc[idx, 'product_virus_ratio_max'] = np.max(temp_ratio)
#         dev_df.loc[idx, 'product_virus_ratio_median'] = np.median(temp_ratio)
#         dev_df.loc[idx, 'product_virus_ratio_std'] = np.std(temp_ratio)
#         dev_df.loc[idx, 'product_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         dev_df.loc[idx, 'product_virus_ratio_avg'] = 0
#         dev_df.loc[idx, 'product_virus_ratio_min'] = 0
#         dev_df.loc[idx, 'product_virus_ratio_max'] = 0
#         dev_df.loc[idx, 'product_virus_ratio_median'] = 0
#         dev_df.loc[idx, 'product_virus_ratio_std'] = 0
#         dev_df.loc[idx, 'product_virus_ratio_rstd'] = 0

#     customer_has_static_df = customer_file_count_df.iloc[temp_count_index, :]
#     dev_df.loc[idx, 'customer_has_static_ratio'] = customer_has_static_df.shape[0] / len(unique_customers)
#     customer_has_static_b3_df = customer_has_static_df[customer_has_static_df['total_event_count']>=3]
#     dev_df.loc[idx, 'customer_has_static_b3_ratio'] = customer_has_static_b3_df.shape[0] / len(unique_customers)

#     dev_df.loc[idx, 'customer_has_static_count'] = customer_has_static_df.shape[0]
#     dev_df.loc[idx, 'customer_has_static_b3_count'] = customer_has_static_b3_df.shape[0]

#     if customer_has_static_df.shape[0] > 0:
#         dev_df.loc[idx, 'customer_virus_ratio_wavg'] = np.average(customer_has_static_df['ratio'], weights=customer_has_static_df['total_event_count'])            
#         dev_df.loc[idx, 'customer_virus_zero_count'] = np.sum(customer_has_static_df['virus_event_count']==0)
#         dev_df.loc[idx, 'customer_virus_zero_ratio'] = dev_df.loc[idx, 'customer_virus_zero_count'] / dev_df.loc[idx, 'affect_user_count']
#         dev_df.loc[idx, 'customer_virus_zero_ratio2'] = dev_df.loc[idx, 'customer_virus_zero_count'] / dev_df.loc[idx, 'customer_has_static_count']

    temp_df['datetime'] = pd.to_datetime(temp_df['QueryTS'], unit='s')
    temp_df['hour'] = temp_df['datetime'].dt.hour
    temp_df['dayofweek'] = temp_df['datetime'].dt.dayofweek
    
    hour_ratio_df = temp_df[['FileID', 'hour']].groupby('hour').count()/temp_df.shape[0]
    dev_df.loc[idx, 'hour_ratio_mean'] = hour_ratio_df['FileID'].mean()
    dev_df.loc[idx, 'hour_ratio_median'] = hour_ratio_df['FileID'].median()
    dev_df.loc[idx, 'hour_ratio_std'] = hour_ratio_df['FileID'].std()
    dev_df.loc[idx, 'hour_ratio_rstd'] = hour_ratio_df['FileID'].std() / (hour_ratio_df['FileID'].mean()+.000000001)
    dev_df.loc[idx, 'hour_ratio_max'] = hour_ratio_df['FileID'].max()
    dev_df.loc[idx, 'hour_ratio_min'] = hour_ratio_df['FileID'].min()
    dev_df.loc[idx, 'hour_occupy_ratio'] = temp_df[['FileID', 'hour']].groupby('hour').count().shape[0]/24.0
    dev_df.loc[idx, 'hour_mean'] = temp_df['hour'].mean()
    
    dayofweek_ratio_df = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count()/temp_df.shape[0]
    dev_df.loc[idx, 'dayofweek_ratio_mean'] = dayofweek_ratio_df['FileID'].mean()
    dev_df.loc[idx, 'dayofweek_ratio_median'] = dayofweek_ratio_df['FileID'].median()
    dev_df.loc[idx, 'dayofweek_ratio_std'] = dayofweek_ratio_df['FileID'].std()
    dev_df.loc[idx, 'dayofweek_ratio_rstd'] = dayofweek_ratio_df['FileID'].std() / (dayofweek_ratio_df['FileID'].mean()+.000000001)
    dev_df.loc[idx, 'dayofweek_ratio_max'] = dayofweek_ratio_df['FileID'].max()
    dev_df.loc[idx, 'dayofweek_ratio_min'] = dayofweek_ratio_df['FileID'].min()
    dev_df.loc[idx, 'dayofweek_occupy_ratio'] = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count().shape[0]/7.0
    dev_df.loc[idx, 'dayofweek_mean'] = temp_df['dayofweek'].mean()
    
    bin_ratio_array = data[['bin_0_ratio', 'bin_1_ratio', 'bin_2_ratio', 'bin_3_ratio', 'bin_4_ratio', \
                            'bin_5_ratio', 'bin_6_ratio', 'bin_7_ratio', 'bin_8_ratio', 'bin_9_ratio']].values
    dev_df.loc[idx, 'bin_ratio_mean'] = np.mean(bin_ratio_array)
    dev_df.loc[idx, 'bin_ratio_median'] = np.median(bin_ratio_array)
    dev_df.loc[idx, 'bin_ratio_std'] = np.std(bin_ratio_array)
    dev_df.loc[idx, 'bin_ratio_rstd'] = np.std(bin_ratio_array) / (np.mean(bin_ratio_array)+.000000001)
    dev_df.loc[idx, 'bin_ratio_max'] = np.max(bin_ratio_array)
    dev_df.loc[idx, 'bin_ratio_min'] = np.min(bin_ratio_array)
    dev_df.loc[idx, 'bin_occupy_ratio'] = np.sum(bin_ratio_array>0)/10.0
    
    the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        dev_df.loc[idx, 'event_diff_time_mean2'] = the_static_df.mean()
        dev_df.loc[idx, 'event_diff_time_median2'] = the_static_df.median()
        dev_df.loc[idx, 'event_diff_time_std2'] = the_static_df.std()
        dev_df.loc[idx, 'event_diff_time_rstd2'] = dev_df.loc[idx, 'event_diff_time_std2'] / (dev_df.loc[idx, 'event_diff_time_mean2']+.000000001)
        dev_df.loc[idx, 'event_diff_time_max2'] = the_static_df.max()
        dev_df.loc[idx, 'event_diff_time_min2'] = the_static_df.min()
        dev_df.loc[idx, 'event_diff_time_lessone_ratio2'] = np.sum(the_static_df<=1) / len(the_static_df)
    
    the_static_df = temp_df.sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        dev_df.loc[idx, 'all_event_diff_time_mean'] = the_static_df.mean()
        dev_df.loc[idx, 'all_event_diff_time_median'] = the_static_df.median()
        dev_df.loc[idx, 'all_event_diff_time_std'] = the_static_df.std()
        dev_df.loc[idx, 'all_event_diff_time_rstd'] = dev_df.loc[idx, 'all_event_diff_time_std'] / (dev_df.loc[idx, 'all_event_diff_time_mean']+.000000001)
        dev_df.loc[idx, 'all_event_diff_time_max'] = the_static_df.max()
        dev_df.loc[idx, 'all_event_diff_time_min'] = the_static_df.min()
        dev_df.loc[idx, 'all_event_diff_time_lessone_ratio'] = np.sum(the_static_df<=1) / len(the_static_df)

#     print(dev_df.loc[idx])
# #     print(temp_df)
# #     print(len(temp_df['CustomerID'].unique()))
# #     print(data)
#     if idx==2:
#         break

100%|██████████| 12810/12810 [40:13<00:00,  4.65it/s] 


In [48]:
dev_df.to_csv('dev_self_0323.csv', index=False)

In [49]:
# test_df['affect_user_count'] = None
# test_df['total_event_count'] = None
# test_df['each_user_event_avg'] = None
# test_df['each_user_event_std'] = None
# test_df['each_user_event_rstd'] = None

# test_df['first_time_occur'] = None
# test_df['last_time_occur'] = None
# test_df['time_duration'] = None

# test_df['first_24h_count'] = None
# test_df['first_24h_ratio'] = None
# test_df['first_24h_affect_user_count'] = None
# test_df['first_24h_affect_user_ratio'] = None
# test_df['first_24h_each_user_event_avg'] = None

# for i in range(0, 10):
#     test_df['bin_'+str(i)+'_count'] = None
#     test_df['bin_'+str(i)+'_ratio'] = None
#     test_df['bin_'+str(i)+'_affect_user_count'] = None
#     test_df['bin_'+str(i)+'_affect_user_ratio'] = None
#     test_df['bin_'+str(i)+'_each_user_event_avg'] = None

# test_df['customer_spread_time_mean'] = None
# test_df['customer_spread_time_std'] = None
# test_df['customer_spread_time_rstd'] = None

# test_df['event_diff_time_mean'] = None
# test_df['event_diff_time_median'] = None
# test_df['event_diff_time_std'] = None
# test_df['event_diff_time_rstd'] = None

# test_df['customer_virus_ratio_avg'] = None
# test_df['customer_virus_ratio_min'] = None
# test_df['customer_virus_ratio_max'] = None
# test_df['customer_virus_ratio_median'] = None
# test_df['customer_virus_ratio_std'] = None
# test_df['customer_virus_ratio_rstd'] = None    

# test_df['customer_virus_ratio_wavg'] = None
# test_df['customer_virus_zero_count'] = None
# test_df['customer_virus_zero_ratio'] = None
# test_df['customer_virus_zero_ratio2'] = None

test_df['hour_ratio_mean'] = None
test_df['hour_ratio_median'] = None
test_df['hour_ratio_std'] = None
test_df['hour_ratio_rstd'] = None
test_df['hour_ratio_max'] = None
test_df['hour_ratio_min'] = None
test_df['hour_occupy_ratio'] = None
test_df['hour_mean'] = None

test_df['dayofweek_ratio_mean'] = None
test_df['dayofweek_ratio_median'] = None
test_df['dayofweek_ratio_std'] = None
test_df['dayofweek_ratio_rstd'] = None
test_df['dayofweek_ratio_max'] = None
test_df['dayofweek_ratio_min'] = None
test_df['dayofweek_occupy_ratio'] = None
test_df['dayofweek_mean'] = None

test_df['bin_ratio_mean'] = None
test_df['bin_ratio_median'] = None
test_df['bin_ratio_std'] = None
test_df['bin_ratio_rstd'] = None
test_df['bin_ratio_max'] = None
test_df['bin_ratio_min'] = None
test_df['bin_occupy_ratio'] = None

test_df['event_diff_time_mean2'] = None
test_df['event_diff_time_median2'] = None
test_df['event_diff_time_std2'] = None
test_df['event_diff_time_rstd2'] = None
test_df['event_diff_time_max2'] = None
test_df['event_diff_time_min2'] = None
test_df['event_diff_time_lessone_ratio2'] = None

test_df['all_event_diff_time_mean'] = None
test_df['all_event_diff_time_median'] = None
test_df['all_event_diff_time_std'] = None
test_df['all_event_diff_time_rstd'] = None
test_df['all_event_diff_time_max'] = None
test_df['all_event_diff_time_min'] = None
test_df['all_event_diff_time_lessone_ratio'] = None

for idx, data in tqdm(test_df.iterrows(), total=len(test_df)):
    temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
#     total_event_count = len(temp_df)
#     test_df.loc[idx, 'affect_user_count'] = len(temp_df['CustomerID'].unique())
#     test_df.loc[idx, 'total_event_count'] = total_event_count
#     test_df.loc[idx, 'each_user_event_avg'] = test_df.loc[idx, 'total_event_count'] / test_df.loc[idx, 'affect_user_count']
#     test_df.loc[idx, 'each_user_event_std'] = temp_df.groupby('CustomerID').count()['QueryTS'].std()
#     test_df.loc[idx, 'each_user_event_rstd'] = test_df.loc[idx, 'each_user_event_std'] / test_df.loc[idx, 'each_user_event_avg']
    
#     time_start = temp_df['QueryTS'].min()
#     time_end = temp_df['QueryTS'].max()
#     time_range = time_end - time_start
    
#     test_df.loc[idx, 'first_time_occur'] = time_start
#     test_df.loc[idx, 'last_time_occur'] = time_end
#     test_df.loc[idx, 'time_duration'] = time_range
    
#     the_flag = ( (temp_df['QueryTS']>=time_start)&(temp_df['QueryTS']<=time_start+86400) )
#     the_24h_count = np.sum(the_flag)
#     test_df.loc[idx, 'first_24h_count'] = the_24h_count
#     test_df.loc[idx, 'first_24h_ratio'] = the_24h_count / total_event_count
#     test_df.loc[idx, 'first_24h_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#     test_df.loc[idx, 'first_24h_affect_user_ratio'] = test_df.loc[idx, 'first_24h_affect_user_count'] / test_df.loc[idx, 'affect_user_count']
#     test_df.loc[idx, 'first_24h_each_user_event_avg'] = the_24h_count / (test_df.loc[idx, 'first_24h_affect_user_count']+.000000001)
    
#     time_step = time_range/10.0
#     for i in range(0, 10):
#         the_flag = ( (temp_df['QueryTS']>=time_start+time_step*i)&(temp_df['QueryTS']<time_start+time_step*(i+1)+0.5) )
#         the_bin_count = np.sum(the_flag)
#         test_df.loc[idx, 'bin_'+str(i)+'_count'] = the_bin_count
#         test_df.loc[idx, 'bin_'+str(i)+'_ratio'] = the_bin_count / total_event_count
#         test_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#         test_df.loc[idx, 'bin_'+str(i)+'_affect_user_ratio'] = test_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] / test_df.loc[idx, 'affect_user_count']
#         test_df.loc[idx, 'bin_'+str(i)+'_each_user_event_avg'] = the_bin_count / (test_df.loc[idx, 'bin_'+str(i)+'_affect_user_count']+.000000001)
    
#     the_static_df = temp_df.groupby('CustomerID').min().sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().fillna(0)
#     test_df.loc[idx, 'customer_spread_time_mean'] = the_static_df.mean()
#     test_df.loc[idx, 'customer_spread_time_std'] = the_static_df.std()
#     test_df.loc[idx, 'customer_spread_time_rstd'] = test_df.loc[idx, 'customer_spread_time_std'] / (test_df.loc[idx, 'customer_spread_time_mean']+.000000001)
    
#     the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().fillna(0)
#     test_df.loc[idx, 'event_diff_time_mean'] = the_static_df.mean()
#     test_df.loc[idx, 'event_diff_time_median'] = the_static_df.median()
#     test_df.loc[idx, 'event_diff_time_std'] = the_static_df.std()
#     test_df.loc[idx, 'event_diff_time_rstd'] = test_df.loc[idx, 'event_diff_time_std'] / (test_df.loc[idx, 'event_diff_time_mean']+.000000001)
    
#     unique_customers = temp_df['CustomerID'].unique().tolist()
#     unique_products = temp_df['ProductID'].unique().tolist()
    
#     temp_ratio = []
#     temp_count_index = []
#     for customer_id in unique_customers:
#         try:
#             found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)
            
#         try:
#             found_count_index = bisect.bisect(customer_file_count_array, customer_id)
#             if customer_file_count_array[found_count_index-1] == customer_id:
#                 temp_count_index.append(found_count_index-1)
#         except:
#             pass
                   
#     if len(temp_ratio) > 0:        
#         test_df.loc[idx, 'customer_virus_ratio_avg'] = np.mean(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_min'] = np.min(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_max'] = np.max(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_median'] = np.median(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_std'] = np.std(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         test_df.loc[idx, 'customer_virus_ratio_avg'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_min'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_max'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_median'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_std'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_rstd'] = 0

#     temp_ratio = []
#     for product_id in unique_products:
#         try:
#             found_ratio = product_virus_ratio_df_group.get_group(product_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)

#     if len(temp_ratio) > 0:
#         test_df.loc[idx, 'product_virus_ratio_avg'] = np.mean(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_min'] = np.min(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_max'] = np.max(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_median'] = np.median(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_std'] = np.std(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         test_df.loc[idx, 'product_virus_ratio_avg'] = 0
#         test_df.loc[idx, 'product_virus_ratio_min'] = 0
#         test_df.loc[idx, 'product_virus_ratio_max'] = 0
#         test_df.loc[idx, 'product_virus_ratio_median'] = 0
#         test_df.loc[idx, 'product_virus_ratio_std'] = 0
#         test_df.loc[idx, 'product_virus_ratio_rstd'] = 0

#     customer_has_static_df = customer_file_count_df.iloc[temp_count_index, :]
#     test_df.loc[idx, 'customer_has_static_ratio'] = customer_has_static_df.shape[0] / len(unique_customers)
#     customer_has_static_b3_df = customer_has_static_df[customer_has_static_df['total_event_count']>=3]
#     test_df.loc[idx, 'customer_has_static_b3_ratio'] = customer_has_static_b3_df.shape[0] / len(unique_customers)

#     test_df.loc[idx, 'customer_has_static_count'] = customer_has_static_df.shape[0]
#     test_df.loc[idx, 'customer_has_static_b3_count'] = customer_has_static_b3_df.shape[0]

#     if customer_has_static_df.shape[0] > 0:
#         test_df.loc[idx, 'customer_virus_ratio_wavg'] = np.average(customer_has_static_df['ratio'], weights=customer_has_static_df['total_event_count'])            
#         test_df.loc[idx, 'customer_virus_zero_count'] = np.sum(customer_has_static_df['virus_event_count']==0)
#         test_df.loc[idx, 'customer_virus_zero_ratio'] = test_df.loc[idx, 'customer_virus_zero_count'] / test_df.loc[idx, 'affect_user_count']
#         test_df.loc[idx, 'customer_virus_zero_ratio2'] = test_df.loc[idx, 'customer_virus_zero_count'] / test_df.loc[idx, 'customer_has_static_count']

    temp_df['datetime'] = pd.to_datetime(temp_df['QueryTS'], unit='s')
    temp_df['hour'] = temp_df['datetime'].dt.hour
    temp_df['dayofweek'] = temp_df['datetime'].dt.dayofweek
    
    hour_ratio_df = temp_df[['FileID', 'hour']].groupby('hour').count()/temp_df.shape[0]
    test_df.loc[idx, 'hour_ratio_mean'] = hour_ratio_df['FileID'].mean()
    test_df.loc[idx, 'hour_ratio_median'] = hour_ratio_df['FileID'].median()
    test_df.loc[idx, 'hour_ratio_std'] = hour_ratio_df['FileID'].std()
    test_df.loc[idx, 'hour_ratio_rstd'] = hour_ratio_df['FileID'].std() / (hour_ratio_df['FileID'].mean()+.000000001)
    test_df.loc[idx, 'hour_ratio_max'] = hour_ratio_df['FileID'].max()
    test_df.loc[idx, 'hour_ratio_min'] = hour_ratio_df['FileID'].min()
    test_df.loc[idx, 'hour_occupy_ratio'] = temp_df[['FileID', 'hour']].groupby('hour').count().shape[0]/24.0
    test_df.loc[idx, 'hour_mean'] = temp_df['hour'].mean()
    
    dayofweek_ratio_df = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count()/temp_df.shape[0]
    test_df.loc[idx, 'dayofweek_ratio_mean'] = dayofweek_ratio_df['FileID'].mean()
    test_df.loc[idx, 'dayofweek_ratio_median'] = dayofweek_ratio_df['FileID'].median()
    test_df.loc[idx, 'dayofweek_ratio_std'] = dayofweek_ratio_df['FileID'].std()
    test_df.loc[idx, 'dayofweek_ratio_rstd'] = dayofweek_ratio_df['FileID'].std() / (dayofweek_ratio_df['FileID'].mean()+.000000001)
    test_df.loc[idx, 'dayofweek_ratio_max'] = dayofweek_ratio_df['FileID'].max()
    test_df.loc[idx, 'dayofweek_ratio_min'] = dayofweek_ratio_df['FileID'].min()
    test_df.loc[idx, 'dayofweek_occupy_ratio'] = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count().shape[0]/7.0
    test_df.loc[idx, 'dayofweek_mean'] = temp_df['dayofweek'].mean()
    
    bin_ratio_array = data[['bin_0_ratio', 'bin_1_ratio', 'bin_2_ratio', 'bin_3_ratio', 'bin_4_ratio', \
                            'bin_5_ratio', 'bin_6_ratio', 'bin_7_ratio', 'bin_8_ratio', 'bin_9_ratio']].values
    test_df.loc[idx, 'bin_ratio_mean'] = np.mean(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_median'] = np.median(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_std'] = np.std(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_rstd'] = np.std(bin_ratio_array) / (np.mean(bin_ratio_array)+.000000001)
    test_df.loc[idx, 'bin_ratio_max'] = np.max(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_min'] = np.min(bin_ratio_array)
    test_df.loc[idx, 'bin_occupy_ratio'] = np.sum(bin_ratio_array>0)/10.0
    
    the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        test_df.loc[idx, 'event_diff_time_mean2'] = the_static_df.mean()
        test_df.loc[idx, 'event_diff_time_median2'] = the_static_df.median()
        test_df.loc[idx, 'event_diff_time_std2'] = the_static_df.std()
        test_df.loc[idx, 'event_diff_time_rstd2'] = test_df.loc[idx, 'event_diff_time_std2'] / (test_df.loc[idx, 'event_diff_time_mean2']+.000000001)
        test_df.loc[idx, 'event_diff_time_max2'] = the_static_df.max()
        test_df.loc[idx, 'event_diff_time_min2'] = the_static_df.min()
        test_df.loc[idx, 'event_diff_time_lessone_ratio2'] = np.sum(the_static_df<=1) / len(the_static_df)
    
    the_static_df = temp_df.sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        test_df.loc[idx, 'all_event_diff_time_mean'] = the_static_df.mean()
        test_df.loc[idx, 'all_event_diff_time_median'] = the_static_df.median()
        test_df.loc[idx, 'all_event_diff_time_std'] = the_static_df.std()
        test_df.loc[idx, 'all_event_diff_time_rstd'] = test_df.loc[idx, 'all_event_diff_time_std'] / (test_df.loc[idx, 'all_event_diff_time_mean']+.000000001)
        test_df.loc[idx, 'all_event_diff_time_max'] = the_static_df.max()
        test_df.loc[idx, 'all_event_diff_time_min'] = the_static_df.min()
        test_df.loc[idx, 'all_event_diff_time_lessone_ratio'] = np.sum(the_static_df<=1) / len(the_static_df)

#     print(test_df.loc[idx])
# #     print(temp_df)
# #     print(len(temp_df['CustomerID'].unique()))
# #     print(data)
#     if idx==2:
#         break

100%|██████████| 6099/6099 [10:13<00:00,  9.94it/s]  


In [50]:
test_df.to_csv('test_self_0323.csv', index=False)

In [21]:
static_column = 'hour_ratio_std'

In [22]:
#training_df[training_df['label']==0][static_column].hist()
print(training_df[training_df['label']==0][static_column].quantile(0.2))
print(training_df[training_df['label']==0][static_column].median())
print(training_df[training_df['label']==0][static_column].quantile(0.8))

0.01580219624297496
0.04051766952827322
0.10247896908228679


In [23]:
#training_df[training_df['label']==1][static_column].hist()
print(training_df[training_df['label']==1][static_column].quantile(0.2))
print(training_df[training_df['label']==1][static_column].median())
print(training_df[training_df['label']==1][static_column].quantile(0.8))

0.03238370335539505
0.10234786407344856
0.29821740045835887


In [15]:
dev_df.shape

(12810, 79)

In [53]:
test_df.columns

Index(['FileID', 'label', 'fold', 'affect_user_count', 'total_event_count',
       'each_user_event_avg', 'each_user_event_std', 'each_user_event_rstd',
       'first_time_occur', 'last_time_occur', 'time_duration',
       'first_24h_count', 'first_24h_ratio', 'first_24h_affect_user_count',
       'first_24h_affect_user_ratio', 'first_24h_each_user_event_avg',
       'bin_0_count', 'bin_0_ratio', 'bin_0_affect_user_count',
       'bin_0_affect_user_ratio', 'bin_0_each_user_event_avg', 'bin_1_count',
       'bin_1_ratio', 'bin_1_affect_user_count', 'bin_1_affect_user_ratio',
       'bin_1_each_user_event_avg', 'bin_2_count', 'bin_2_ratio',
       'bin_2_affect_user_count', 'bin_2_affect_user_ratio',
       'bin_2_each_user_event_avg', 'bin_3_count', 'bin_3_ratio',
       'bin_3_affect_user_count', 'bin_3_affect_user_ratio',
       'bin_3_each_user_event_avg', 'bin_4_count', 'bin_4_ratio',
       'bin_4_affect_user_count', 'bin_4_affect_user_ratio',
       'bin_4_each_user_event_avg', 'b

In [75]:
training_df = training_df.replace([np.inf, -np.inf], np.nan)
dev_df = dev_df.replace([np.inf, -np.inf], np.nan)
test_df = test_df.replace([np.inf, -np.inf], np.nan)

training_df = training_df.fillna(0)
dev_df = dev_df.fillna(0)
test_df = test_df.fillna(0)

In [124]:
from sklearn.linear_model import Ridge
from sklearn import preprocessing

In [163]:
scaler = preprocessing.StandardScaler().fit(dev_df.drop(drop_features, axis=1))
drop_features = ['FileID', 'label', 'fold', 'first_time_occur', 'last_time_occur']

def transform_x(df, scaler):
    temp_df = scaler.transform(df)
    
    new_X = temp_df
    
    #new_X = np.hstack((temp_df**(i+1) for i in range(2)))
    
    #poly_pre = preprocessing.PolynomialFeatures(2)
    #new_X = poly_pre.fit_transform(temp_df)
    
    return new_X

In [164]:
logreg = Ridge()
logreg.fit(transform_x(dev_df.drop(drop_features, axis=1), scaler), dev_df['label'])

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [165]:
fpr, tpr, thresholds = roc_curve(dev_df['label'], logreg.predict(transform_x(dev_df.drop(drop_features, axis=1), scaler)))
print(auc(fpr, tpr))
fpr, tpr, thresholds = roc_curve(test_df['label'], logreg.predict(transform_x(test_df.drop(drop_features, axis=1), scaler)))
print(auc(fpr, tpr))

0.931629065091
0.919433111909


In [166]:
print('Feature names:', training_df.drop(drop_features, axis=1).columns)
print('Feature importances:', list(logreg.coef_))

[(coe, x) for coe,x in sorted(zip(list(np.abs(logreg.coef_)), training_df.drop(drop_features, axis=1).columns.tolist()))]

Feature names: Index(['affect_user_count', 'total_event_count', 'each_user_event_avg',
       'each_user_event_std', 'each_user_event_rstd', 'time_duration',
       'first_24h_count', 'first_24h_ratio', 'first_24h_affect_user_count',
       'first_24h_affect_user_ratio', 'first_24h_each_user_event_avg',
       'bin_0_count', 'bin_0_ratio', 'bin_0_affect_user_count',
       'bin_0_affect_user_ratio', 'bin_0_each_user_event_avg', 'bin_1_count',
       'bin_1_ratio', 'bin_1_affect_user_count', 'bin_1_affect_user_ratio',
       'bin_1_each_user_event_avg', 'bin_2_count', 'bin_2_ratio',
       'bin_2_affect_user_count', 'bin_2_affect_user_ratio',
       'bin_2_each_user_event_avg', 'bin_3_count', 'bin_3_ratio',
       'bin_3_affect_user_count', 'bin_3_affect_user_ratio',
       'bin_3_each_user_event_avg', 'bin_4_count', 'bin_4_ratio',
       'bin_4_affect_user_count', 'bin_4_affect_user_ratio',
       'bin_4_each_user_event_avg', 'bin_5_count', 'bin_5_ratio',
       'bin_5_affect_user_coun

[(0.00025934502343010878, 'bin_2_count'),
 (0.00030109675490372476, 'bin_7_ratio'),
 (0.0006331761029968818, 'each_user_event_rstd'),
 (0.00075102105599252739, 'total_event_count'),
 (0.00077546190371664012, 'bin_5_each_user_event_avg'),
 (0.0012562549845749785, 'bin_3_ratio'),
 (0.0013848234778078782, 'bin_4_ratio'),
 (0.0014990742065190484, 'bin_2_affect_user_ratio'),
 (0.0017940602199800547, 'bin_7_affect_user_count'),
 (0.0019787930779798443, 'bin_7_count'),
 (0.0021348709524885196, 'bin_8_ratio'),
 (0.0023835322970573616, 'customer_virus_ratio_min'),
 (0.0025329866159879057, 'bin_3_affect_user_ratio'),
 (0.0026644608638316828, 'bin_3_each_user_event_avg'),
 (0.0027798067807746217, 'event_diff_time_rstd'),
 (0.0034731466169998534, 'bin_9_ratio'),
 (0.003629856099329708, 'event_diff_time_median'),
 (0.0037172948089799801, 'first_24h_each_user_event_avg'),
 (0.0037501552080087975, 'bin_6_ratio'),
 (0.0047910550106330984, 'bin_8_affect_user_ratio'),
 (0.005805895291584662, 'bin_8_each

In [93]:
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(dev_df.drop(drop_features, axis=1), dev_df['label'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [94]:
fpr, tpr, thresholds = roc_curve(test_df['label'], logreg.predict_proba(test_df.drop(drop_features, axis=1))[:, 1])
print(auc(fpr, tpr))

0.81428451437


In [108]:
from sklearn.svm import SVC

In [112]:
logreg = SVC(class_weight='balanced', probability=True)
logreg.fit(dev_df.drop(drop_features, axis=1), dev_df['label'])

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [113]:
fpr, tpr, thresholds = roc_curve(test_df['label'], logreg.predict_proba(test_df.drop(drop_features, axis=1))[:, 1])
print(auc(fpr, tpr))

0.49620298731


In [95]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [104]:
logreg = DecisionTreeClassifier(max_depth=5, min_samples_leaf=10)
logreg.fit(dev_df.drop(drop_features, axis=1), dev_df['label'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [105]:
fpr, tpr, thresholds = roc_curve(dev_df['label'], logreg.predict_proba(dev_df.drop(drop_features, axis=1))[:, 1])
print(auc(fpr, tpr))
fpr, tpr, thresholds = roc_curve(test_df['label'], logreg.predict_proba(test_df.drop(drop_features, axis=1))[:, 1])
print(auc(fpr, tpr))

0.940571350592
0.917181950391


In [106]:
result_df['dt'] = logreg.predict_proba(test_df.drop(drop_features, axis=1))[:, 1]

In [107]:
result_df.corr()

,ridge,dt
ridge,1.000000,0.869876
dt,0.869876,1.000000


In [4]:
training_df = pd.read_csv('training-set-fold.csv')
test_df = pd.read_csv('testing-set.csv', header=None, names=['FileID', 'label'])
train_exception_df = pd.read_csv('./exception/exception_train.txt', header=None, names=['FileID'])
test_exception_df = pd.read_csv('./exception/exception_testing.txt', header=None, names=['FileID'])

training_df = training_df[~training_df['FileID'].isin(train_exception_df['FileID'])]
test_df = test_df[~test_df['FileID'].isin(test_exception_df['FileID'])]

print(training_df.shape)
print(test_df.shape)

(52518, 3)
(29376, 2)


In [52]:
training_df = pd.read_csv('df_log/228/training_0314.csv')
test_df = pd.read_csv('df_log/228/test_0314.csv')

In [32]:
training_five_folds = []
for idx, fold in enumerate(five_folds):
    temp_df = pd.read_csv('df_log/228/training_0314_fold'+str(idx)+'.csv')
    training_five_folds.append(temp_df)

In [37]:
training_df = pd.concat([training_five_folds[0], training_five_folds[1], training_five_folds[2], training_five_folds[3], training_five_folds[4]]).reset_index()

In [28]:
customer_virus_ratio_df = pd.read_csv('from_table/228/real/customer_virus_ratio.csv')
customer_virus_ratio_df_group = customer_virus_ratio_df.groupby('t1_CustomerID')

product_virus_ratio_df = pd.read_csv('from_table/228/real/product_virus_ratio.csv')
product_virus_ratio_df_group = product_virus_ratio_df.groupby('t1_ProductID')

customer_file_count_df = pd.read_csv('from_table/228/real/customer_file_count.csv')
customer_file_count_array = customer_file_count_df['t2_CustomerID'].tolist()
customer_file_count_array.sort()

customer_virus_ratio_df_temp = customer_virus_ratio_df.copy()
customer_virus_ratio_df_temp['t2_CustomerID'] = customer_virus_ratio_df_temp['t1_CustomerID']
customer_virus_ratio_df_temp = customer_virus_ratio_df_temp.drop('t1_CustomerID', axis=1)
customer_file_count_df = customer_file_count_df.merge(customer_virus_ratio_df_temp.drop('total_event_count', axis=1), on='t2_CustomerID', how='left')
customer_file_count_df = customer_file_count_df.fillna(0)
customer_file_count_df['ratio'] = customer_file_count_df['virus_event_count'] / (customer_file_count_df['total_event_count']+.000000001)
customer_file_count_df = customer_file_count_df.sort_values('t2_CustomerID')
customer_file_count_df = customer_file_count_df.reset_index()
del customer_virus_ratio_df_temp

print(customer_file_count_df.head())
customer_file_count_array = customer_file_count_df['t2_CustomerID'].tolist()
customer_file_count_array.sort()
print(customer_file_count_array[0:10])

     index                     t2_CustomerID  total_event_count  ratio  \
0   458708  0000006fa286976bf35ea17f1f19bc7a                  1    0.0   
1  2193096  00000145d9062eada528bace5fb4864e                  2    0.0   
2    29299  000003d77bee692d83b7b83fdba60d3d                 24    0.0   
3  1749482  00000aea7887eaf96dbd022025e426fd                  1    0.0   
4   893851  0000109c46b9b8922f1ce2aaa684442f                  1    0.0   

   virus_event_count  
0                0.0  
1                0.0  
2                0.0  
3                0.0  
4                0.0  
['0000006fa286976bf35ea17f1f19bc7a', '00000145d9062eada528bace5fb4864e', '000003d77bee692d83b7b83fdba60d3d', '00000aea7887eaf96dbd022025e426fd', '0000109c46b9b8922f1ce2aaa684442f', '0000110a6b286c121488a19f7f5c0ad0', '0000119795c2f643a5111de4a0f135c6', '000012cb18f09736997b05c334799ff6', '0000136b39e3e7892157113d03e1cac8', '00001d108011435a6bc742400a8e6abd']


In [35]:
customer_file_count_df[customer_file_count_df['ratio'] != 0]

,index,t2_CustomerID,total_event_count,ratio,virus_event_count
192,2589640,0002c3abe241025876525dcddda40d29,2,0.500000,1.0
204,3021341,0002f27f7b4661cf3414059c775f83ef,3,1.000000,3.0
273,4195808,0003d97857d32fed9f7425043d79b0cf,14,1.000000,14.0
291,2356412,00040a6f8e309c574572f8498f5ef61b,2,0.500000,1.0
331,2691459,0004aef88b3db0bf64d7618983234f05,2,0.500000,1.0
332,2762829,0004aefde2280e9fc41f368c0472cdeb,3,0.666667,2.0
354,1928502,000523c4654f5784cae047ef4d406cdc,1,1.000000,1.0
409,2440655,0005dda4e2de09748a94fea6cd819bf1,2,0.500000,1.0
420,3208525,0005fb25265277c4c51c97d3b44a948e,4,0.250000,1.0
459,291851,00067b2b989aebe5725ba0fee554fbd9,22,0.045455,1.0


In [38]:
training_df.head()

,index,FileID,fold,label,affect_user_count,total_event_count,each_user_event_avg,each_user_event_std,each_user_event_rstd,first_time_occur,last_time_occur,time_duration,first_24h_count,first_24h_ratio,first_24h_affect_user_count,first_24h_affect_user_ratio,first_24h_each_user_event_avg,bin_0_count,bin_0_ratio,bin_0_affect_user_count,bin_0_affect_user_ratio,bin_0_each_user_event_avg,bin_1_count,bin_1_ratio,bin_1_affect_user_count,bin_1_affect_user_ratio,bin_1_each_user_event_avg,bin_2_count,bin_2_ratio,bin_2_affect_user_count,bin_2_affect_user_ratio,bin_2_each_user_event_avg,bin_3_count,bin_3_ratio,bin_3_affect_user_count,bin_3_affect_user_ratio,bin_3_each_user_event_avg,bin_4_count,bin_4_ratio,bin_4_affect_user_count,bin_4_affect_user_ratio,bin_4_each_user_event_avg,bin_5_count,bin_5_ratio,bin_5_affect_user_count,bin_5_affect_user_ratio,bin_5_each_user_event_avg,bin_6_count,bin_6_ratio,bin_6_affect_user_count,bin_6_affect_user_ratio,bin_6_each_user_event_avg,bin_7_count,bin_7_ratio,bin_7_affect_user_count,bin_7_affect_user_ratio,bin_7_each_user_event_avg,bin_8_count,bin_8_ratio,bin_8_affect_user_count,bin_8_affect_user_ratio,bin_8_each_user_event_avg,bin_9_count,bin_9_ratio,bin_9_affect_user_count,bin_9_affect_user_ratio,bin_9_each_user_event_avg,customer_spread_time_mean,customer_spread_time_std,customer_spread_time_rstd,event_diff_time_mean,event_diff_time_median,event_diff_time_std,event_diff_time_rstd,customer_virus_ratio_avg,customer_virus_ratio_min,customer_virus_ratio_max,customer_virus_ratio_median,customer_virus_ratio_std,customer_virus_ratio_rstd,product_virus_ratio_avg,product_virus_ratio_min,product_virus_ratio_max,product_virus_ratio_median,product_virus_ratio_std,product_virus_ratio_rstd,customer_has_static_ratio,customer_has_static_b3_ratio,customer_has_static_count,customer_has_static_b3_count,customer_virus_ratio_wavg,customer_virus_zero_count,customer_virus_zero_ratio,customer_virus_zero_ratio2
0,0,cc22fa47ac06f6f0813379b3fe4fa252,4,1,22,87,3.954545,8.357929,2.113499,1491630510,1492088818,458308,11,0.126437,4,0.181818,2.750000,10,0.114943,3,0.136364,3.333333,1,0.011494,1,0.045455,1.00,6,0.068966,4,0.181818,1.50,8,0.091954,1,0.045455,8.000000,6,0.068966,5,0.227273,1.200000,51,0.586207,6,0.272727,8.500000,2,0.022989,2,0.090909,1.000000,1,0.011494,1,0.045455,1.0,0,0.000000,0,0.000000,0.0,2,0.022989,1,0.045455,2.0,15198.000000,15170.078723,0.998163,4993.218391,2.0,37880.054476,7.586300,0.590798,0.000000,1.000000,0.775,0.392383,0.664158,0.116088,0.000000,0.173516,0.127157,0.052315,0.450648,0.863636,0.636364,19.0,14.0,0.157634,0.0,0.000000,0.000000
1,1,cc2c2dd7bfbfde8e2f0658f27aa5f082,4,1,15,31,2.066667,2.086236,1.009469,1489577110,1489810151,233041,30,0.967742,15,1.000000,2.000000,27,0.870968,14,0.933333,1.928571,2,0.064516,2,0.133333,1.00,0,0.000000,0,0.000000,0.00,1,0.032258,1,0.066667,1.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.0,0,0.000000,0,0.000000,0.0,1,0.032258,1,0.066667,1.0,2273.533333,8554.817981,3.762785,11706.580645,1.0,43469.005530,3.713211,0.930498,0.160124,1.000000,1.000,0.214218,0.230219,0.116708,0.000000,0.163860,0.152322,0.061785,0.529399,1.000000,0.933333,15.0,14.0,0.182990,0.0,0.000000,0.000000
2,2,cc2fbf1cc2671ccb3a6b8443b2d1bf25,4,1,3,395,131.666667,114.369285,0.868627,1488367460,1488466168,98708,389,0.984810,3,1.000000,129.666667,78,0.197468,1,0.333333,78.000000,251,0.635443,2,0.666667,125.50,60,0.151899,1,0.333333,60.00,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,0,0.000000,0,0.000000,0.0,0,0.000000,0,0.000000,0.0,6,0.015190,1,0.333333,6.0,4675.666667,5519.297449,1.180430,246.015190,31.0,4139.997071,16.828217,0.250000,0.000000,0.500000,0.250,0.250000,1.000000,0.062810,0.000000,0.107236,0.081193,0.045668,0.727085,0.666667,0.333333,2.0,1.0,0.600000,0.0,0.000000,0.000000
3,3,cc3405ec030a2479bc88669333fe8f15,4,1,156,336,2.153846,5.838

In [58]:
# training_df['affect_user_count'] = None
# training_df['total_event_count'] = None
# training_df['each_user_event_avg'] = None
# training_df['each_user_event_std'] = None
# training_df['each_user_event_rstd'] = None

# training_df['first_time_occur'] = None
# training_df['last_time_occur'] = None
# training_df['time_duration'] = None

# training_df['first_24h_count'] = None
# training_df['first_24h_ratio'] = None
# training_df['first_24h_affect_user_count'] = None
# training_df['first_24h_affect_user_ratio'] = None
# training_df['first_24h_each_user_event_avg'] = None

# for i in range(0, 10):
#     training_df['bin_'+str(i)+'_count'] = None
#     training_df['bin_'+str(i)+'_ratio'] = None
#     training_df['bin_'+str(i)+'_affect_user_count'] = None
#     training_df['bin_'+str(i)+'_affect_user_ratio'] = None
#     training_df['bin_'+str(i)+'_each_user_event_avg'] = None

# training_df['customer_spread_time_mean'] = None
# training_df['customer_spread_time_std'] = None
# training_df['customer_spread_time_rstd'] = None

# training_df['event_diff_time_mean'] = None
# training_df['event_diff_time_median'] = None
# training_df['event_diff_time_std'] = None
# training_df['event_diff_time_rstd'] = None

# training_df['customer_virus_ratio_avg'] = None
# training_df['customer_virus_ratio_min'] = None
# training_df['customer_virus_ratio_max'] = None
# training_df['customer_virus_ratio_median'] = None
# training_df['customer_virus_ratio_std'] = None
# training_df['customer_virus_ratio_rstd'] = None  

# training_df['product_virus_ratio_avg'] = None
# training_df['product_virus_ratio_min'] = None
# training_df['product_virus_ratio_max'] = None
# training_df['product_virus_ratio_median'] = None
# training_df['product_virus_ratio_std'] = None
# training_df['product_virus_ratio_rstd'] = None  

training_df['hour_ratio_mean'] = None
training_df['hour_ratio_median'] = None
training_df['hour_ratio_std'] = None
training_df['hour_ratio_rstd'] = None
training_df['hour_ratio_max'] = None
training_df['hour_ratio_min'] = None
training_df['hour_occupy_ratio'] = None
training_df['hour_mean'] = None

training_df['dayofweek_ratio_mean'] = None
training_df['dayofweek_ratio_median'] = None
training_df['dayofweek_ratio_std'] = None
training_df['dayofweek_ratio_rstd'] = None
training_df['dayofweek_ratio_max'] = None
training_df['dayofweek_ratio_min'] = None
training_df['dayofweek_occupy_ratio'] = None
training_df['dayofweek_mean'] = None

training_df['bin_ratio_mean'] = None
training_df['bin_ratio_median'] = None
training_df['bin_ratio_std'] = None
training_df['bin_ratio_rstd'] = None
training_df['bin_ratio_max'] = None
training_df['bin_ratio_min'] = None
training_df['bin_occupy_ratio'] = None

training_df['event_diff_time_mean2'] = None
training_df['event_diff_time_median2'] = None
training_df['event_diff_time_std2'] = None
training_df['event_diff_time_rstd2'] = None
training_df['event_diff_time_max2'] = None
training_df['event_diff_time_min2'] = None
training_df['event_diff_time_lessone_ratio2'] = None

training_df['all_event_diff_time_mean'] = None
training_df['all_event_diff_time_median'] = None
training_df['all_event_diff_time_std'] = None
training_df['all_event_diff_time_rstd'] = None
training_df['all_event_diff_time_max'] = None
training_df['all_event_diff_time_min'] = None
training_df['all_event_diff_time_lessone_ratio'] = None

for idx, data in tqdm(training_df.iterrows(), total=len(training_df)):
    temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
    total_event_count = len(temp_df)
#     training_df.loc[idx, 'affect_user_count'] = len(temp_df['CustomerID'].unique())
#     training_df.loc[idx, 'total_event_count'] = total_event_count
#     training_df.loc[idx, 'each_user_event_avg'] = training_df.loc[idx, 'total_event_count'] / training_df.loc[idx, 'affect_user_count']
#     training_df.loc[idx, 'each_user_event_std'] = temp_df.groupby('CustomerID').count()['QueryTS'].std()
#     training_df.loc[idx, 'each_user_event_rstd'] = training_df.loc[idx, 'each_user_event_std'] / training_df.loc[idx, 'each_user_event_avg']
    
#     time_start = temp_df['QueryTS'].min()
#     time_end = temp_df['QueryTS'].max()
#     time_range = time_end - time_start
    
#     training_df.loc[idx, 'first_time_occur'] = time_start
#     training_df.loc[idx, 'last_time_occur'] = time_end
#     training_df.loc[idx, 'time_duration'] = time_range
    
#     the_flag = ( (temp_df['QueryTS']>=time_start)&(temp_df['QueryTS']<=time_start+86400) )
#     the_24h_count = np.sum(the_flag)
#     training_df.loc[idx, 'first_24h_count'] = the_24h_count
#     training_df.loc[idx, 'first_24h_ratio'] = the_24h_count / total_event_count
#     training_df.loc[idx, 'first_24h_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#     training_df.loc[idx, 'first_24h_affect_user_ratio'] = training_df.loc[idx, 'first_24h_affect_user_count'] / training_df.loc[idx, 'affect_user_count']
#     training_df.loc[idx, 'first_24h_each_user_event_avg'] = the_24h_count / (training_df.loc[idx, 'first_24h_affect_user_count']+.000000001)
    
#     time_step = time_range/10.0
#     for i in range(0, 10):
#         the_flag = ( (temp_df['QueryTS']>=time_start+time_step*i)&(temp_df['QueryTS']<time_start+time_step*(i+1)+0.5) )
#         the_bin_count = np.sum(the_flag)
#         training_df.loc[idx, 'bin_'+str(i)+'_count'] = the_bin_count
#         training_df.loc[idx, 'bin_'+str(i)+'_ratio'] = the_bin_count / total_event_count
#         training_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#         training_df.loc[idx, 'bin_'+str(i)+'_affect_user_ratio'] = training_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] / training_df.loc[idx, 'affect_user_count']
#         training_df.loc[idx, 'bin_'+str(i)+'_each_user_event_avg'] = the_bin_count / (training_df.loc[idx, 'bin_'+str(i)+'_affect_user_count']+.000000001)
    
#     the_static_df = temp_df.groupby('CustomerID').min().sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().fillna(0)
#     training_df.loc[idx, 'customer_spread_time_mean'] = the_static_df.mean()
#     training_df.loc[idx, 'customer_spread_time_std'] = the_static_df.std()
#     training_df.loc[idx, 'customer_spread_time_rstd'] = training_df.loc[idx, 'customer_spread_time_std'] / (training_df.loc[idx, 'customer_spread_time_mean']+.000000001)
    
#     the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().fillna(0)
#     training_df.loc[idx, 'event_diff_time_mean'] = the_static_df.mean()
#     training_df.loc[idx, 'event_diff_time_median'] = the_static_df.median()
#     training_df.loc[idx, 'event_diff_time_std'] = the_static_df.std()
#     training_df.loc[idx, 'event_diff_time_rstd'] = training_df.loc[idx, 'event_diff_time_std'] / (training_df.loc[idx, 'event_diff_time_mean']+.000000001)
    
#     unique_customers = temp_df['CustomerID'].unique().tolist()
#     temp_ratio = []
#     temp_count_index = []
#     for customer_id in unique_customers:
#         try:
#             found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)
            
#         try:
#             found_count_index = bisect.bisect(customer_file_count_array, customer_id)
#             if customer_file_count_array[found_count_index-1] == customer_id:
#                 temp_count_index.append(found_count_index-1)
#         except:
#             pass
        
#     if len(temp_ratio) > 0:        
#         training_df.loc[idx, 'customer_virus_ratio_avg'] = np.mean(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_min'] = np.min(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_max'] = np.max(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_median'] = np.median(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_std'] = np.std(temp_ratio)
#         training_df.loc[idx, 'customer_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         training_df.loc[idx, 'customer_virus_ratio_avg'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_min'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_max'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_median'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_std'] = 0
#         training_df.loc[idx, 'customer_virus_ratio_rstd'] = 0
    
#     unique_products = temp_df['ProductID'].unique().tolist()
#     temp_ratio = []
#     for product_id in unique_products:
#         try:
#             found_ratio = product_virus_ratio_df_group.get_group(product_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)

#     if len(temp_ratio) > 0:
#         training_df.loc[idx, 'product_virus_ratio_avg'] = np.mean(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_min'] = np.min(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_max'] = np.max(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_median'] = np.median(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_std'] = np.std(temp_ratio)
#         training_df.loc[idx, 'product_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         training_df.loc[idx, 'product_virus_ratio_avg'] = 0
#         training_df.loc[idx, 'product_virus_ratio_min'] = 0
#         training_df.loc[idx, 'product_virus_ratio_max'] = 0
#         training_df.loc[idx, 'product_virus_ratio_median'] = 0
#         training_df.loc[idx, 'product_virus_ratio_std'] = 0
#         training_df.loc[idx, 'product_virus_ratio_rstd'] = 0
    
#     customer_has_static_df = customer_file_count_df.iloc[temp_count_index, :]
#     training_df.loc[idx, 'customer_has_static_ratio'] = customer_has_static_df.shape[0] / len(unique_customers)
#     customer_has_static_b3_df = customer_has_static_df[customer_has_static_df['total_event_count']>=3]
#     training_df.loc[idx, 'customer_has_static_b3_ratio'] = customer_has_static_b3_df.shape[0] / len(unique_customers)

#     training_df.loc[idx, 'customer_has_static_count'] = customer_has_static_df.shape[0]
#     training_df.loc[idx, 'customer_has_static_b3_count'] = customer_has_static_b3_df.shape[0]

#     if customer_has_static_df.shape[0] > 0:
#         training_df.loc[idx, 'customer_virus_ratio_wavg'] = np.average(customer_has_static_df['ratio'], weights=customer_has_static_df['total_event_count'])            
#         training_df.loc[idx, 'customer_virus_zero_count'] = np.sum(customer_has_static_df['virus_event_count']==0)
#         training_df.loc[idx, 'customer_virus_zero_ratio'] = training_df.loc[idx, 'customer_virus_zero_count'] / training_df.loc[idx, 'affect_user_count']
#         training_df.loc[idx, 'customer_virus_zero_ratio2'] = training_df.loc[idx, 'customer_virus_zero_count'] / training_df.loc[idx, 'customer_has_static_count']

    temp_df['datetime'] = pd.to_datetime(temp_df['QueryTS'], unit='s')
    temp_df['hour'] = temp_df['datetime'].dt.hour
    temp_df['dayofweek'] = temp_df['datetime'].dt.dayofweek
    
    hour_ratio_df = temp_df[['FileID', 'hour']].groupby('hour').count()/temp_df.shape[0]
    training_df.loc[idx, 'hour_ratio_mean'] = hour_ratio_df['FileID'].mean()
    training_df.loc[idx, 'hour_ratio_median'] = hour_ratio_df['FileID'].median()
    training_df.loc[idx, 'hour_ratio_std'] = hour_ratio_df['FileID'].std()
    training_df.loc[idx, 'hour_ratio_rstd'] = hour_ratio_df['FileID'].std() / (hour_ratio_df['FileID'].mean()+.000000001)
    training_df.loc[idx, 'hour_ratio_max'] = hour_ratio_df['FileID'].max()
    training_df.loc[idx, 'hour_ratio_min'] = hour_ratio_df['FileID'].min()
    training_df.loc[idx, 'hour_occupy_ratio'] = temp_df[['FileID', 'hour']].groupby('hour').count().shape[0]/24.0
    training_df.loc[idx, 'hour_mean'] = temp_df['hour'].mean()
    
    dayofweek_ratio_df = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count()/temp_df.shape[0]
    training_df.loc[idx, 'dayofweek_ratio_mean'] = dayofweek_ratio_df['FileID'].mean()
    training_df.loc[idx, 'dayofweek_ratio_median'] = dayofweek_ratio_df['FileID'].median()
    training_df.loc[idx, 'dayofweek_ratio_std'] = dayofweek_ratio_df['FileID'].std()
    training_df.loc[idx, 'dayofweek_ratio_rstd'] = dayofweek_ratio_df['FileID'].std() / (dayofweek_ratio_df['FileID'].mean()+.000000001)
    training_df.loc[idx, 'dayofweek_ratio_max'] = dayofweek_ratio_df['FileID'].max()
    training_df.loc[idx, 'dayofweek_ratio_min'] = dayofweek_ratio_df['FileID'].min()
    training_df.loc[idx, 'dayofweek_occupy_ratio'] = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count().shape[0]/7.0
    training_df.loc[idx, 'dayofweek_mean'] = temp_df['dayofweek'].mean()
    
    bin_ratio_array = data[['bin_0_ratio', 'bin_1_ratio', 'bin_2_ratio', 'bin_3_ratio', 'bin_4_ratio', \
                            'bin_5_ratio', 'bin_6_ratio', 'bin_7_ratio', 'bin_8_ratio', 'bin_9_ratio']].values
    training_df.loc[idx, 'bin_ratio_mean'] = np.mean(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_median'] = np.median(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_std'] = np.std(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_rstd'] = np.std(bin_ratio_array) / (np.mean(bin_ratio_array)+.000000001)
    training_df.loc[idx, 'bin_ratio_max'] = np.max(bin_ratio_array)
    training_df.loc[idx, 'bin_ratio_min'] = np.min(bin_ratio_array)
    training_df.loc[idx, 'bin_occupy_ratio'] = np.sum(bin_ratio_array>0)/10.0
    
    the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        training_df.loc[idx, 'event_diff_time_mean2'] = the_static_df.mean()
        training_df.loc[idx, 'event_diff_time_median2'] = the_static_df.median()
        training_df.loc[idx, 'event_diff_time_std2'] = the_static_df.std()
        training_df.loc[idx, 'event_diff_time_rstd2'] = training_df.loc[idx, 'event_diff_time_std2'] / (training_df.loc[idx, 'event_diff_time_mean2']+.000000001)
        training_df.loc[idx, 'event_diff_time_max2'] = the_static_df.max()
        training_df.loc[idx, 'event_diff_time_min2'] = the_static_df.min()
        training_df.loc[idx, 'event_diff_time_lessone_ratio2'] = np.sum(the_static_df<=1) / len(the_static_df)
    
    the_static_df = temp_df.sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        training_df.loc[idx, 'all_event_diff_time_mean'] = the_static_df.mean()
        training_df.loc[idx, 'all_event_diff_time_median'] = the_static_df.median()
        training_df.loc[idx, 'all_event_diff_time_std'] = the_static_df.std()
        training_df.loc[idx, 'all_event_diff_time_rstd'] = training_df.loc[idx, 'all_event_diff_time_std'] / (training_df.loc[idx, 'all_event_diff_time_mean']+.000000001)
        training_df.loc[idx, 'all_event_diff_time_max'] = the_static_df.max()
        training_df.loc[idx, 'all_event_diff_time_min'] = the_static_df.min()
        training_df.loc[idx, 'all_event_diff_time_lessone_ratio'] = np.sum(the_static_df<=1) / len(the_static_df)

    print(training_df.loc[idx])
#     print(temp_df)
#     print(len(temp_df['CustomerID'].unique()))
#     print(data)
    if idx==2:
        break

  0%|          | 1/52518 [00:00<10:32:59,  1.38it/s]

FileID                               0000e2398b12121a85166fed5fe2a3da
label                                                               0
affect_user_count                                                   3
total_event_count                                                  47
each_user_event_avg                                           15.6667
each_user_event_std                                           19.3477
each_user_event_rstd                                          1.23496
first_time_occur                                           1488608668
last_time_occur                                            1488982004
time_duration                                                  373336
first_24h_count                                                    46
first_24h_ratio                                              0.978723
first_24h_affect_user_count                                         3
first_24h_affect_user_ratio                                         1
first_24h_each_user_

In [ ]:
training_df.to_csv('training_0323.csv', index=False)

In [56]:
# test_df['affect_user_count'] = None
# test_df['total_event_count'] = None
# test_df['each_user_event_avg'] = None
# test_df['each_user_event_std'] = None
# test_df['each_user_event_rstd'] = None

# test_df['first_time_occur'] = None
# test_df['last_time_occur'] = None
# test_df['time_duration'] = None

# test_df['first_24h_count'] = None
# test_df['first_24h_ratio'] = None
# test_df['first_24h_affect_user_count'] = None
# test_df['first_24h_affect_user_ratio'] = None
# test_df['first_24h_each_user_event_avg'] = None

# for i in range(0, 10):
#     test_df['bin_'+str(i)+'_count'] = None
#     test_df['bin_'+str(i)+'_ratio'] = None
#     test_df['bin_'+str(i)+'_affect_user_count'] = None
#     test_df['bin_'+str(i)+'_affect_user_ratio'] = None
#     test_df['bin_'+str(i)+'_each_user_event_avg'] = None

# test_df['customer_spread_time_mean'] = None
# test_df['customer_spread_time_std'] = None
# test_df['customer_spread_time_rstd'] = None

# test_df['event_diff_time_mean'] = None
# test_df['event_diff_time_median'] = None
# test_df['event_diff_time_std'] = None
# test_df['event_diff_time_rstd'] = None

# test_df['customer_virus_ratio_avg'] = None
# test_df['customer_virus_ratio_min'] = None
# test_df['customer_virus_ratio_max'] = None
# test_df['customer_virus_ratio_median'] = None
# test_df['customer_virus_ratio_std'] = None
# test_df['customer_virus_ratio_rstd'] = None  

# test_df['product_virus_ratio_avg'] = None
# test_df['product_virus_ratio_min'] = None
# test_df['product_virus_ratio_max'] = None
# test_df['product_virus_ratio_median'] = None
# test_df['product_virus_ratio_std'] = None
# test_df['product_virus_ratio_rstd'] = None  

test_df['hour_ratio_mean'] = None
test_df['hour_ratio_median'] = None
test_df['hour_ratio_std'] = None
test_df['hour_ratio_rstd'] = None
test_df['hour_ratio_max'] = None
test_df['hour_ratio_min'] = None
test_df['hour_occupy_ratio'] = None
test_df['hour_mean'] = None

test_df['dayofweek_ratio_mean'] = None
test_df['dayofweek_ratio_median'] = None
test_df['dayofweek_ratio_std'] = None
test_df['dayofweek_ratio_rstd'] = None
test_df['dayofweek_ratio_max'] = None
test_df['dayofweek_ratio_min'] = None
test_df['dayofweek_occupy_ratio'] = None
test_df['dayofweek_mean'] = None

test_df['bin_ratio_mean'] = None
test_df['bin_ratio_median'] = None
test_df['bin_ratio_std'] = None
test_df['bin_ratio_rstd'] = None
test_df['bin_ratio_max'] = None
test_df['bin_ratio_min'] = None
test_df['bin_occupy_ratio'] = None

test_df['event_diff_time_mean2'] = None
test_df['event_diff_time_median2'] = None
test_df['event_diff_time_std2'] = None
test_df['event_diff_time_rstd2'] = None
test_df['event_diff_time_max2'] = None
test_df['event_diff_time_min2'] = None
test_df['event_diff_time_lessone_ratio2'] = None

test_df['all_event_diff_time_mean'] = None
test_df['all_event_diff_time_median'] = None
test_df['all_event_diff_time_std'] = None
test_df['all_event_diff_time_rstd'] = None
test_df['all_event_diff_time_max'] = None
test_df['all_event_diff_time_min'] = None
test_df['all_event_diff_time_lessone_ratio'] = None

for idx, data in tqdm(test_df.iterrows(), total=len(test_df)):
    temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
    total_event_count = len(temp_df)
#     test_df.loc[idx, 'affect_user_count'] = len(temp_df['CustomerID'].unique())
#     test_df.loc[idx, 'total_event_count'] = total_event_count
#     test_df.loc[idx, 'each_user_event_avg'] = test_df.loc[idx, 'total_event_count'] / test_df.loc[idx, 'affect_user_count']
#     test_df.loc[idx, 'each_user_event_std'] = temp_df.groupby('CustomerID').count()['QueryTS'].std()
#     test_df.loc[idx, 'each_user_event_rstd'] = test_df.loc[idx, 'each_user_event_std'] / test_df.loc[idx, 'each_user_event_avg']
    
#     time_start = temp_df['QueryTS'].min()
#     time_end = temp_df['QueryTS'].max()
#     time_range = time_end - time_start
    
#     test_df.loc[idx, 'first_time_occur'] = time_start
#     test_df.loc[idx, 'last_time_occur'] = time_end
#     test_df.loc[idx, 'time_duration'] = time_range
    
#     the_flag = ( (temp_df['QueryTS']>=time_start)&(temp_df['QueryTS']<=time_start+86400) )
#     the_24h_count = np.sum(the_flag)
#     test_df.loc[idx, 'first_24h_count'] = the_24h_count
#     test_df.loc[idx, 'first_24h_ratio'] = the_24h_count / total_event_count
#     test_df.loc[idx, 'first_24h_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#     test_df.loc[idx, 'first_24h_affect_user_ratio'] = test_df.loc[idx, 'first_24h_affect_user_count'] / test_df.loc[idx, 'affect_user_count']
#     test_df.loc[idx, 'first_24h_each_user_event_avg'] = the_24h_count / (test_df.loc[idx, 'first_24h_affect_user_count']+.000000001)
    
#     time_step = time_range/10.0
#     for i in range(0, 10):
#         the_flag = ( (temp_df['QueryTS']>=time_start+time_step*i)&(temp_df['QueryTS']<time_start+time_step*(i+1)+0.5) )
#         the_bin_count = np.sum(the_flag)
#         test_df.loc[idx, 'bin_'+str(i)+'_count'] = the_bin_count
#         test_df.loc[idx, 'bin_'+str(i)+'_ratio'] = the_bin_count / total_event_count
#         test_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] = len(temp_df[the_flag]['CustomerID'].unique())
#         test_df.loc[idx, 'bin_'+str(i)+'_affect_user_ratio'] = test_df.loc[idx, 'bin_'+str(i)+'_affect_user_count'] / test_df.loc[idx, 'affect_user_count']
#         test_df.loc[idx, 'bin_'+str(i)+'_each_user_event_avg'] = the_bin_count / (test_df.loc[idx, 'bin_'+str(i)+'_affect_user_count']+.000000001)
    
#     the_static_df = temp_df.groupby('CustomerID').min().sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().fillna(0)
#     test_df.loc[idx, 'customer_spread_time_mean'] = the_static_df.mean()
#     test_df.loc[idx, 'customer_spread_time_std'] = the_static_df.std()
#     test_df.loc[idx, 'customer_spread_time_rstd'] = test_df.loc[idx, 'customer_spread_time_std'] / (test_df.loc[idx, 'customer_spread_time_mean']+.000000001)
    
#     the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().fillna(0)
#     test_df.loc[idx, 'event_diff_time_mean'] = the_static_df.mean()
#     test_df.loc[idx, 'event_diff_time_median'] = the_static_df.median()
#     test_df.loc[idx, 'event_diff_time_std'] = the_static_df.std()
#     test_df.loc[idx, 'event_diff_time_rstd'] = test_df.loc[idx, 'event_diff_time_std'] / (test_df.loc[idx, 'event_diff_time_mean']+.000000001)
    
#     unique_customers = temp_df['CustomerID'].unique().tolist()
#     temp_ratio = []
#     temp_count_index = []
#     for customer_id in unique_customers:
#         try:
#             found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)
            
#         try:
#             found_count_index = bisect.bisect(customer_file_count_array, customer_id)
#             if customer_file_count_array[found_count_index-1] == customer_id:
#                 temp_count_index.append(found_count_index-1)
#         except:
#             pass
        
#     if len(temp_ratio) > 0:        
#         test_df.loc[idx, 'customer_virus_ratio_avg'] = np.mean(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_min'] = np.min(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_max'] = np.max(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_median'] = np.median(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_std'] = np.std(temp_ratio)
#         test_df.loc[idx, 'customer_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         test_df.loc[idx, 'customer_virus_ratio_avg'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_min'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_max'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_median'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_std'] = 0
#         test_df.loc[idx, 'customer_virus_ratio_rstd'] = 0
    
#     unique_products = temp_df['ProductID'].unique().tolist()
#     temp_ratio = []
#     for product_id in unique_products:
#         try:
#             found_ratio = product_virus_ratio_df_group.get_group(product_id).iloc[0]
#             if found_ratio['total_event_count'] >= 3:
#                 temp_ratio.append(found_ratio['ratio'])
#         except:
#             temp_ratio.append(0)

#     if len(temp_ratio) > 0:
#         test_df.loc[idx, 'product_virus_ratio_avg'] = np.mean(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_min'] = np.min(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_max'] = np.max(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_median'] = np.median(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_std'] = np.std(temp_ratio)
#         test_df.loc[idx, 'product_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
#     else:
#         test_df.loc[idx, 'product_virus_ratio_avg'] = 0
#         test_df.loc[idx, 'product_virus_ratio_min'] = 0
#         test_df.loc[idx, 'product_virus_ratio_max'] = 0
#         test_df.loc[idx, 'product_virus_ratio_median'] = 0
#         test_df.loc[idx, 'product_virus_ratio_std'] = 0
#         test_df.loc[idx, 'product_virus_ratio_rstd'] = 0
        
#     customer_has_static_df = customer_file_count_df.iloc[temp_count_index, :]
#     test_df.loc[idx, 'customer_has_static_ratio'] = customer_has_static_df.shape[0] / len(unique_customers)
#     customer_has_static_b3_df = customer_has_static_df[customer_has_static_df['total_event_count']>=3]
#     test_df.loc[idx, 'customer_has_static_b3_ratio'] = customer_has_static_b3_df.shape[0] / len(unique_customers)

#     test_df.loc[idx, 'customer_has_static_count'] = customer_has_static_df.shape[0]
#     test_df.loc[idx, 'customer_has_static_b3_count'] = customer_has_static_b3_df.shape[0]

#     if customer_has_static_df.shape[0] > 0:
#         test_df.loc[idx, 'customer_virus_ratio_wavg'] = np.average(customer_has_static_df['ratio'], weights=customer_has_static_df['total_event_count'])            
#         test_df.loc[idx, 'customer_virus_zero_count'] = np.sum(customer_has_static_df['virus_event_count']==0)
#         test_df.loc[idx, 'customer_virus_zero_ratio'] = test_df.loc[idx, 'customer_virus_zero_count'] / test_df.loc[idx, 'affect_user_count']
#         test_df.loc[idx, 'customer_virus_zero_ratio2'] = test_df.loc[idx, 'customer_virus_zero_count'] / test_df.loc[idx, 'customer_has_static_count']

    temp_df['datetime'] = pd.to_datetime(temp_df['QueryTS'], unit='s')
    temp_df['hour'] = temp_df['datetime'].dt.hour
    temp_df['dayofweek'] = temp_df['datetime'].dt.dayofweek
    
    hour_ratio_df = temp_df[['FileID', 'hour']].groupby('hour').count()/temp_df.shape[0]
    test_df.loc[idx, 'hour_ratio_mean'] = hour_ratio_df['FileID'].mean()
    test_df.loc[idx, 'hour_ratio_median'] = hour_ratio_df['FileID'].median()
    test_df.loc[idx, 'hour_ratio_std'] = hour_ratio_df['FileID'].std()
    test_df.loc[idx, 'hour_ratio_rstd'] = hour_ratio_df['FileID'].std() / (hour_ratio_df['FileID'].mean()+.000000001)
    test_df.loc[idx, 'hour_ratio_max'] = hour_ratio_df['FileID'].max()
    test_df.loc[idx, 'hour_ratio_min'] = hour_ratio_df['FileID'].min()
    test_df.loc[idx, 'hour_occupy_ratio'] = temp_df[['FileID', 'hour']].groupby('hour').count().shape[0]/24.0
    test_df.loc[idx, 'hour_mean'] = temp_df['hour'].mean()
    
    dayofweek_ratio_df = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count()/temp_df.shape[0]
    test_df.loc[idx, 'dayofweek_ratio_mean'] = dayofweek_ratio_df['FileID'].mean()
    test_df.loc[idx, 'dayofweek_ratio_median'] = dayofweek_ratio_df['FileID'].median()
    test_df.loc[idx, 'dayofweek_ratio_std'] = dayofweek_ratio_df['FileID'].std()
    test_df.loc[idx, 'dayofweek_ratio_rstd'] = dayofweek_ratio_df['FileID'].std() / (dayofweek_ratio_df['FileID'].mean()+.000000001)
    test_df.loc[idx, 'dayofweek_ratio_max'] = dayofweek_ratio_df['FileID'].max()
    test_df.loc[idx, 'dayofweek_ratio_min'] = dayofweek_ratio_df['FileID'].min()
    test_df.loc[idx, 'dayofweek_occupy_ratio'] = temp_df[['FileID', 'dayofweek']].groupby('dayofweek').count().shape[0]/7.0
    test_df.loc[idx, 'dayofweek_mean'] = temp_df['dayofweek'].mean()
    
    bin_ratio_array = data[['bin_0_ratio', 'bin_1_ratio', 'bin_2_ratio', 'bin_3_ratio', 'bin_4_ratio', \
                            'bin_5_ratio', 'bin_6_ratio', 'bin_7_ratio', 'bin_8_ratio', 'bin_9_ratio']].values
    test_df.loc[idx, 'bin_ratio_mean'] = np.mean(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_median'] = np.median(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_std'] = np.std(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_rstd'] = np.std(bin_ratio_array) / (np.mean(bin_ratio_array)+.000000001)
    test_df.loc[idx, 'bin_ratio_max'] = np.max(bin_ratio_array)
    test_df.loc[idx, 'bin_ratio_min'] = np.min(bin_ratio_array)
    test_df.loc[idx, 'bin_occupy_ratio'] = np.sum(bin_ratio_array>0)/10.0
    
    the_static_df = temp_df.sort_values(['CustomerID', 'QueryTS'], ascending=True).groupby(['CustomerID'])['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        test_df.loc[idx, 'event_diff_time_mean2'] = the_static_df.mean()
        test_df.loc[idx, 'event_diff_time_median2'] = the_static_df.median()
        test_df.loc[idx, 'event_diff_time_std2'] = the_static_df.std()
        test_df.loc[idx, 'event_diff_time_rstd2'] = test_df.loc[idx, 'event_diff_time_std2'] / (test_df.loc[idx, 'event_diff_time_mean2']+.000000001)
        test_df.loc[idx, 'event_diff_time_max2'] = the_static_df.max()
        test_df.loc[idx, 'event_diff_time_min2'] = the_static_df.min()
        test_df.loc[idx, 'event_diff_time_lessone_ratio2'] = np.sum(the_static_df<=1) / len(the_static_df)
    
    the_static_df = temp_df.sort_values(['QueryTS'], ascending=True)['QueryTS'].diff().dropna()
    if the_static_df.shape[0] > 0:
        test_df.loc[idx, 'all_event_diff_time_mean'] = the_static_df.mean()
        test_df.loc[idx, 'all_event_diff_time_median'] = the_static_df.median()
        test_df.loc[idx, 'all_event_diff_time_std'] = the_static_df.std()
        test_df.loc[idx, 'all_event_diff_time_rstd'] = test_df.loc[idx, 'all_event_diff_time_std'] / (test_df.loc[idx, 'all_event_diff_time_mean']+.000000001)
        test_df.loc[idx, 'all_event_diff_time_max'] = the_static_df.max()
        test_df.loc[idx, 'all_event_diff_time_min'] = the_static_df.min()
        test_df.loc[idx, 'all_event_diff_time_lessone_ratio'] = np.sum(the_static_df<=1) / len(the_static_df)
        
#     print(test_df.loc[idx])
# #     print(temp_df)
# #     print(len(temp_df['CustomerID'].unique()))
# #     print(data)
#     if idx==2:
#         break


100%|██████████| 29376/29376 [1:27:51<00:00,  5.37it/s]  


In [57]:
test_df.to_csv('test_0323.csv', index=False)

In [23]:
training_df.head()

,FileID,label,affect_user_count,total_event_count,each_user_event_avg,each_user_event_std,each_user_event_rstd,first_time_occur,last_time_occur,time_duration,...,product_virus_ratio_avg,product_virus_ratio_min,product_virus_ratio_max,product_virus_ratio_median,product_virus_ratio_std,product_virus_ratio_rstd,customer_has_static_ratio,customer_has_static_b3_ratio,customer_has_static_count,customer_has_static_b3_count
0,0000e2398b12121a85166fed5fe2a3da,0,3,47,15.666667,19.347696,1.234959,1488608668,1488982004,373336,...,0.093069,0.08063,0.105508,0.093069,0.012439,0.133652,1.0,0.333333,3.0,1.0
1,0001fe8dce14ce099aa6ca8ea5026ea7,0,234,234,1.000000,0.000000,0.000000,1489595016,1490052955,457939,...,0.060526,0.00000,0.105508,0.068299,0.039089,0.645815,1.0,0.482906,234.0,113.0
2,00027f50019000accc492e5684efc818,0,22,378,17.181818,32.871520,1.913157,1491346740,1491661407,314667,...,0.107731,0.00000,0.160751,0.123216,0.056041,0.520190,1.0,0.363636,22.0,8.0
3,00028c9da3573ec50db74b44310ae507,0,113,368,3.256637,6.078032,1.866352,1492215082,1492699191,484109,...,0.097563,0.00000,0.160751,0.105508,0.056109,0.575111,1.0,0.504425,113.0,57.0
4,0003dc8130969abe688cadf5f14ea19f,0,130,235,1.807692,3.320353,1.836791,1491267443,1491727740,460297,...,0.080571,0.00000,0.160751,0.080630,0.053193,0.660204,1.0,0.538462,130.0,70.0


In [24]:
test_df.head()

,FileID,label,affect_user_count,total_event_count,each_user_event_avg,each_user_event_std,each_user_event_rstd,first_time_occur,last_time_occur,time_duration,...,product_virus_ratio_avg,product_virus_ratio_min,product_virus_ratio_max,product_virus_ratio_median,product_virus_ratio_std,product_virus_ratio_rstd,customer_has_static_ratio,customer_has_static_b3_ratio,customer_has_static_count,customer_has_static_b3_count
0,00008c73ee43c15b16c26b26398c1577,0.5,91,134,1.472527,1.447915,0.983286,1494826826,1495312972,486146,...,0.154023,0.105508,0.176752,0.160751,0.025086,0.162875,0.593407,0.318681,54.0,29.0
1,0002ded3a0b54f2ffdab0ca77a5ce2b6,0.5,5,791,158.200000,157.742512,0.997108,1494145742,1494340930,195188,...,0.133130,0.105508,0.160751,0.133130,0.027622,0.207479,0.800000,0.600000,4.0,3.0
2,00050a9df8e13f6ab5a3d3b3e2fc6a86,0.5,614,617,1.004886,0.069786,0.069446,1494387611,1494892592,504981,...,0.060526,0.000000,0.105508,0.068299,0.039089,0.645815,0.791531,0.377850,486.0,232.0
3,000b1aa62b95e448784b8b341de46c64,0.5,177,242,1.367232,0.549431,0.401857,1495860195,1496232359,372164,...,0.107314,0.013889,0.183162,0.105508,0.050435,0.469979,0.847458,0.418079,150.0,74.0
4,000d9f96b5eddf04a3b7a37cb95d0a00,0.5,29,73,2.517241,2.680894,1.065013,1494324090,1494798251,474161,...,0.094087,0.000000,0.176752,0.105508,0.072609,0.771727,0.344828,0.137931,10.0,4.0


In [3]:
five_folds = [[[0, 1, 2, 3], [4]], [[0, 1, 2, 4], [3]], [[0, 1, 3, 4], [2]], [[0, 2, 3, 4], [1]], [[1, 2, 3, 4], [0]]]

In [4]:
from functools import reduce

temp_dfs = []

for i in range(0, 5):
    temp_dfs.append(pd.read_csv('from_table/228/real/customer_virus_ratio_fold'+str(i)+'.csv').set_index('t1_CustomerID'))

# print(temp_dfs[0].head())
# print(temp_dfs[1].head())
# print(temp_dfs[0].add(temp_dfs[1], fill_value=0).head())

customer_virus_ratio_dfs = []
customer_virus_ratio_df_groups = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    temp_df = reduce(lambda left, right: left.add(right, fill_value=0), \
                     [temp_dfs[merge_folds[0]], temp_dfs[merge_folds[1]], temp_dfs[merge_folds[2]], temp_dfs[merge_folds[3]]])
    temp_df['ratio'] = temp_df['virus_event_count'] / temp_df['total_event_count']
    #print(temp_df[['virus_event_count', 'total_event_count']])
    customer_virus_ratio_dfs.append(temp_df.reset_index())
    customer_virus_ratio_df_groups.append(customer_virus_ratio_dfs[idx].groupby('t1_CustomerID'))
    #print(customer_virus_ratio_dfs[idx])
    print(idx)
    print(fold)
    

temp_dfs = []

for i in range(0, 5):
    temp_dfs.append(pd.read_csv('from_table/228/real/product_virus_ratio_fold'+str(i)+'.csv').set_index('t1_ProductID'))

# print(temp_dfs[0].head())
# print(temp_dfs[1].head())
# print(temp_dfs[0].add(temp_dfs[1], fill_value=0).head())

product_virus_ratio_dfs = []
product_virus_ratio_df_groups = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    temp_df = reduce(lambda left, right: left.add(right, fill_value=0), \
                     [temp_dfs[merge_folds[0]], temp_dfs[merge_folds[1]], temp_dfs[merge_folds[2]], temp_dfs[merge_folds[3]]])
    temp_df['ratio'] = temp_df['virus_event_count'] / temp_df['total_event_count']
    #print(temp_df[['virus_event_count', 'total_event_count']])
    product_virus_ratio_dfs.append(temp_df.reset_index())
    product_virus_ratio_df_groups.append(product_virus_ratio_dfs[idx].groupby('t1_ProductID'))
    #print(product_virus_ratio_dfs[idx])
    print(idx)
    print(fold)
    

temp_dfs = []

for i in range(0, 5):
    customer_virus_ratio_df = pd.read_csv('from_table/228/real/customer_virus_ratio_fold'+str(i)+'.csv')
    customer_file_count_df = pd.read_csv('from_table/228/real/customer_file_count_fold'+str(i)+'.csv')
    
    customer_virus_ratio_df['t2_CustomerID'] = customer_virus_ratio_df['t1_CustomerID']
    customer_virus_ratio_df = customer_virus_ratio_df.drop('t1_CustomerID', axis=1)
    
    customer_file_count_df = customer_file_count_df.merge(customer_virus_ratio_df.drop('total_event_count', axis=1), on='t2_CustomerID', how='left')
    customer_file_count_df = customer_file_count_df.fillna(0)
    temp_dfs.append(customer_file_count_df.set_index('t2_CustomerID'))

# print(temp_dfs[0].head())
# print(temp_dfs[1].head())
# print(temp_dfs[0].add(temp_dfs[1], fill_value=0).head())

customer_file_count_dfs = []
for idx, fold in enumerate(five_folds):
    merge_folds = fold[0]
    temp_df = reduce(lambda left, right: left.add(right, fill_value=0), \
                     [temp_dfs[merge_folds[0]], temp_dfs[merge_folds[1]], temp_dfs[merge_folds[2]], temp_dfs[merge_folds[3]]])
    #print(temp_df[['virus_event_count', 'total_event_count']])
    temp_df['ratio'] = temp_df['virus_event_count'] / (temp_df['total_event_count']+.000000001)
    customer_file_count_dfs.append(temp_df.reset_index())
    #print(product_virus_ratio_dfs[idx])
    print(idx)
    print(fold)

0
[[0, 1, 2, 3], [4]]
1
[[0, 1, 2, 4], [3]]
2
[[0, 1, 3, 4], [2]]
3
[[0, 2, 3, 4], [1]]
4
[[1, 2, 3, 4], [0]]
0
[[0, 1, 2, 3], [4]]
1
[[0, 1, 2, 4], [3]]
2
[[0, 1, 3, 4], [2]]
3
[[0, 2, 3, 4], [1]]
4
[[1, 2, 3, 4], [0]]
0
[[0, 1, 2, 3], [4]]
1
[[0, 1, 2, 4], [3]]
2
[[0, 1, 3, 4], [2]]
3
[[0, 2, 3, 4], [1]]
4
[[1, 2, 3, 4], [0]]


In [5]:
training_df = pd.read_csv('df_log/228/training_0314.csv')
training_df_with_fold = pd.read_csv('training-set-fold.csv')
print(training_df_with_fold.shape)
print(training_df.shape)
training_df = training_df_with_fold[['FileID', 'fold']].merge(training_df, on='FileID', how='inner')
print(training_df.shape)

(52518, 3)
(52518, 92)
(52518, 93)


In [6]:
training_five_folds = []

for fold_idx, fold in enumerate(five_folds):
    customer_virus_ratio_df_group = customer_virus_ratio_df_groups[fold_idx]
    product_virus_ratio_df_group = product_virus_ratio_df_groups[fold_idx]
    customer_file_count_df = customer_file_count_dfs[fold_idx].sort_values('t2_CustomerID')
    customer_file_count_array = customer_file_count_df['t2_CustomerID'].tolist()
    customer_file_count_array.sort()
    
    
    fold_training = training_df[training_df['fold'].isin(fold[1])].copy()
    
    for data_idx, data in tqdm(fold_training.iterrows(), total=len(fold_training)):
        temp_df = pd.read_csv('file_log/'+data['FileID']+'.csv', header=None, names=['FileID', 'CustomerID', 'QueryTS', 'ProductID'])
        unique_customers = temp_df['CustomerID'].unique().tolist()
        unique_products = temp_df['ProductID'].unique().tolist()
        
        temp_ratio = []
        temp_count_index = []
        for customer_id in unique_customers:
            try:
                found_ratio = customer_virus_ratio_df_group.get_group(customer_id).iloc[0]
                if found_ratio['total_event_count'] >= 3:
                    temp_ratio.append(found_ratio['ratio'])
            except:
                temp_ratio.append(0)
                
            try:
                found_count_index = bisect.bisect(customer_file_count_array, customer_id)
                if customer_file_count_array[found_count_index-1] == customer_id:
                    temp_count_index.append(found_count_index-1)
            except:
                pass
                
        if len(temp_ratio) > 0:        
            fold_training.loc[data_idx, 'customer_virus_ratio_avg'] = np.mean(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_min'] = np.min(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_max'] = np.max(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_median'] = np.median(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_std'] = np.std(temp_ratio)
            fold_training.loc[data_idx, 'customer_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
        else:
            fold_training.loc[data_idx, 'customer_virus_ratio_avg'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_min'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_max'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_median'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_std'] = 0
            fold_training.loc[data_idx, 'customer_virus_ratio_rstd'] = 0
            
        temp_ratio = []
        for product_id in unique_products:
            try:
                found_ratio = product_virus_ratio_df_group.get_group(product_id).iloc[0]
                if found_ratio['total_event_count'] >= 3:
                    temp_ratio.append(found_ratio['ratio'])
            except:
                temp_ratio.append(0)
                
        if len(temp_ratio) > 0:        
            fold_training.loc[data_idx, 'product_virus_ratio_avg'] = np.mean(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_min'] = np.min(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_max'] = np.max(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_median'] = np.median(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_std'] = np.std(temp_ratio)
            fold_training.loc[data_idx, 'product_virus_ratio_rstd'] = np.std(temp_ratio) / (np.mean(temp_ratio)+.000000001)
        else:
            fold_training.loc[data_idx, 'product_virus_ratio_avg'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_min'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_max'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_median'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_std'] = 0
            fold_training.loc[data_idx, 'product_virus_ratio_rstd'] = 0
        
        customer_has_static_df = customer_file_count_df.iloc[temp_count_index, :]
        fold_training.loc[data_idx, 'customer_has_static_ratio'] = customer_has_static_df.shape[0] / len(unique_customers)
        customer_has_static_b3_df = customer_has_static_df[customer_has_static_df['total_event_count']>=3]
        fold_training.loc[data_idx, 'customer_has_static_b3_ratio'] = customer_has_static_b3_df.shape[0] / len(unique_customers)

        fold_training.loc[data_idx, 'customer_has_static_count'] = customer_has_static_df.shape[0]
        fold_training.loc[data_idx, 'customer_has_static_b3_count'] = customer_has_static_b3_df.shape[0]

        if customer_has_static_df.shape[0] > 0:
            fold_training.loc[data_idx, 'customer_virus_ratio_wavg'] = np.average(customer_has_static_df['ratio'], weights=customer_has_static_df['total_event_count'])            
            fold_training.loc[data_idx, 'customer_virus_zero_count'] = np.sum(customer_has_static_df['virus_event_count']==0)
            fold_training.loc[data_idx, 'customer_virus_zero_ratio'] = fold_training.loc[data_idx, 'customer_virus_zero_count'] / fold_training.loc[data_idx, 'affect_user_count']
            fold_training.loc[data_idx, 'customer_virus_zero_ratio2'] = fold_training.loc[data_idx, 'customer_virus_zero_count'] / fold_training.loc[data_idx, 'customer_has_static_count']
        
#         print(fold_training.loc[data_idx])
#         break
    
    #training_five_folds.append(fold_training)
    print(fold_training.head())
    training_five_folds.append(fold_training)
    fold_training.to_csv('training_0314_fold'+str(fold_idx)+'.csv', index=False)
    

100%|██████████| 10503/10503 [07:17<00:00, 24.00it/s]


                                 FileID  fold  label  affect_user_count  \
26841  cc22fa47ac06f6f0813379b3fe4fa252     4      1                 22   
26847  cc2c2dd7bfbfde8e2f0658f27aa5f082     4      1                 15   
26849  cc2fbf1cc2671ccb3a6b8443b2d1bf25     4      1                  3   
26852  cc3405ec030a2479bc88669333fe8f15     4      1                156   
26859  cc39f4ca9e62c6a4edfd79bda5d8da52     4      1                  7   

       total_event_count  each_user_event_avg  each_user_event_std  \
26841                 87             3.954545             8.357929   
26847                 31             2.066667             2.086236   
26849                395           131.666667           114.369285   
26852                336             2.153846             5.838862   
26859                 31             4.428571             5.191568   

       each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
26841              2.113499        1491630510  

100%|██████████| 10503/10503 [07:08<00:00, 24.52it/s]


                                 FileID  fold  label  affect_user_count  \
20158  98e6408d996cf68b2c18b32e451b6b23     3      0                 23   
20159  98e6d86eea7114f4012cc93ac4cfebb5     3      0                 31   
20160  98e7d6284ae3903e9ac4f331215121a9     3      0                671   
20161  98ea5e3f8cf10f9dea74db353603e6dd     3      0                164   
20162  98eb30382fda57947d293ff078cdd440     3      0                642   

       total_event_count  each_user_event_avg  each_user_event_std  \
20158                188             8.173913            10.002964   
20159                 71             2.290323             1.883145   
20160                733             1.092399             0.289805   
20161               1889            11.518293            49.377036   
20162                642             1.000000             0.000000   

       each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
20158              1.223767        1489971944  

100%|██████████| 10504/10504 [06:50<00:00, 25.56it/s]


                                 FileID  fold  label  affect_user_count  \
13414  65ab4cace270f44f11086cfecefc9ed7     2      0                 62   
13415  65adc239f58279f60080bd37a807cd79     2      0                 20   
13416  65afdf087f1f54836d7a7bcbb675700a     2      0                  8   
13417  65b0615861a338268e67956354e9900d     2      0                 15   
13418  65b22629e9c6b7e7ff2f7f131fba07c0     2      0                 20   

       total_event_count  each_user_event_avg  each_user_event_std  \
13414                103             1.661290             0.957174   
13415                704            35.200000           148.713996   
13416                 66             8.250000             7.166390   
13417                310            20.666667            29.553019   
13418                 83             4.150000             4.233637   

       each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
13414              0.576163        1489213092  

100%|██████████| 10504/10504 [06:54<00:00, 27.70it/s]


                                FileID  fold  label  affect_user_count  \
6744  32af84f8ff7cd0651650bc74579e409e     1      0                129   
6745  32b0b03e84c99675a18e99fe23385b66     1      0                332   
6746  32b2ae7cd9903db0a1600d5b041c81e7     1      0                  2   
6747  32b4086bbee1f5e725ec512c6f2de0f7     1      0              10092   
6748  32b6c20a07a7c83da93c0b4dd17395de     1      0               3868   

      total_event_count  each_user_event_avg  each_user_event_std  \
6744                178             1.379845             1.596842   
6745                333             1.003012             0.054882   
6746                 66            33.000000            32.526912   
6747              10092             1.000000             0.000000   
6748               3891             1.005946             0.076892   

      each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
6744              1.157262        1491610775       149212768

100%|██████████| 10504/10504 [06:38<00:00, 40.27it/s]


                             FileID  fold  label  affect_user_count  \
0  0000e2398b12121a85166fed5fe2a3da     0      0                  3   
1  0001fe8dce14ce099aa6ca8ea5026ea7     0      0                234   
2  00027f50019000accc492e5684efc818     0      0                 22   
3  0003dc8130969abe688cadf5f14ea19f     0      0                130   
4  0006ac9aadbdbf2fed5e62b4e463daa1     0      0                 19   

   total_event_count  each_user_event_avg  each_user_event_std  \
0                 47            15.666667            19.347696   
1                234             1.000000             0.000000   
2                378            17.181818            32.871520   
3                235             1.807692             3.320353   
4                101             5.315789             5.676977   

   each_user_event_rstd  first_time_occur  last_time_occur  time_duration  \
0              1.234959        1488608668       1488982004         373336   
1              0.00000